# Facility Booking List for FBS Booking (Ver 3.0)

**Objective**: Produce a populated Facility Booking List with all cells in Text Format 

**Overview**: 
- Assign a data frame *(a table like structure with rows and columns to manipulate data)* to progressively obtain the necessary column values from the various excel sheets provided
- Throughout the process of obtaining values, data processing and cleaning is needed to obtain the values in the desired format. 
- The data frame will be then used to populate the Facility Booking List.

**Step-by-Step Approach**:
1. Load all excel files require to obtain the necessary values
    - Read and combine all the gvSession files into a singular dataframe
2. Filter Out Necessary Columns in gvSession, and for courses that are not conducted within SMU, upload the data to separate sheet in the Facility Booking List template file
    - "Sch#" column will also be extracted from the "gvSession" file
3. Obtain Time From and Time To Values from gvSession
    - Perform data cleaning to reflect the requirements for the Booking Start Time and End Times
4. Obtain Use Type Column by referencing if courses have an IO code or not in (Latest) SSG Approved_Master Listing 
    - If have, "Event". If not, "Adhoc"
5. Obtain Purpose Column 
6. Obtain Event Code Column by referencing if courses have an IO code or not in (Latest) SSG Approved_Master Listing 
    - Event Code will be the IO Code
7. Obtain Facility Name, Building, and Floor columns
8. Remove any unnecessary columns, keeping only the columns needed in the Facility Booking List file
9. Perform all additional checks
    - For courses that run for more than 1 day, assign the same venue
    - Check for venue clashes and see if 2 or more courses have the same venue, and correct accordingly
    - Book Catering for the respective venues
    - Generate the "Venue Preference" Column
    - Generate the "No.of Course Days" Column to indicate the run duration for each course, ie. 1, 2, 3 days or "Above 3 Days"
    - If course code cannot be found in FBS Report: Indicate "Assignment of Venue required" under "Venue" Column
    - Ensure that all the facility names follow the naming convention suitable for FBS, using Facility Names File to look up the correct naming  
10. Final Touches
    - Rename "Date of Booking", "Time Booking From" and "Time Booking To" to required format
    - Generate the "Any Comments" column for users to indicate additional comments
    - Sort the dataframe to group courses together, by "Purpose" and "Date of Booking"
    - Change the format of all column values to text format
11. Populate Facility+Booking+List.xlsx with the final DataFrame


## 1.0 Import python libraries & read all datasets and files

In [1]:
import pandas as pd
import numpy as np
import warnings
import os
import glob
import datetime
import openpyxl


from dateutil.parser import parse, ParserError
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter
from datetime import datetime, timedelta

#Ignore warnings
warnings.filterwarnings("ignore")

### 1.1 Verify The Current Working Directory, ie. the location where all files are placed

In [2]:
# Check current directory
print(os.getcwd())  # Verify you are in /content

c:\Users\somes\OneDrive\Desktop\PT Job Materials\FBS Pre-Booking Code\Pre-FBE Python Code 3.0


### 1.2 Import All Required Datasets 

- **FBS Report:** BKG01+-+Details+of+Facilities+Booking+by+Individual's+Organization+Unit_Cost+Centre.xlsx
- **TMS File:** gvSession.xlsx
- **SMUA Venue Matters File:** SMUA Venue matters.xlsx
- **Facility Names File:** Facility Names.xlsx
- **SSG Master Listing File:** (Latest) SSG Approved_Master Listing.xlsx
- **Facility Booking File:** Facility+Booking+List.xlsx (Will be used later)

In [3]:
# gvSession
gvSession_initial_df = pd.read_excel('gvSession.xlsx')
print(gvSession_initial_df.shape)

# Rooms Booked over FBS, remove last row and remove first 10 rows
fbs_report_df = pd.read_excel("BKG01+-+Details+of+Facilities+Booking+by+Individual's+Organization+Unit_Cost+Centre.xlsx", skiprows=9, skipfooter=1)

# # SMUA Venue Matters
pillar_room_pref_df = pd.read_excel('SMUA Venue matters.xlsx', sheet_name=None, skiprows=1)

# SSG Master Listing, read the "Master" Sheet into the dataframe
ssg_master_listing_df = pd.read_excel('(Latest) SSG Approved_Master Listing.xlsx', sheet_name='Master')

# Facility Names
facility_names_df = pd.read_excel('Facility Names.xlsx')

print("All Files Loaded Succesfully!")

(1000, 22)
All Files Loaded Succesfully!


### 1.3 Merge All "gvSession" Files

1. There are multiple gvSession files that reside in the same directory as the python code. We need to identify all of these files, read all of these files into Python, and combine them into a singular gvSession dataframe. 

2. The method for identifying these files in the current directory is to check if the "gvSession" prefix exists in the filename, and then concatenates these files together

In [4]:
#get the current working directory
current_directory = os.getcwd()
directory_path = current_directory
file_prefix = "gvSession"
file_paths = glob.glob(os.path.join(directory_path, f"{file_prefix}*.xlsx"))

dataframes = []

# Iterate through the file paths and read Excel files
for file_path in file_paths:
    try:
        df = pd.read_excel(file_path)
        dataframes.append(df)
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")

# Concatenate all dataframes if there are any
if dataframes:
    gvSession_df = pd.concat(dataframes, ignore_index=True)
    print("Concatenation successful.")
else:
    print("No files found matching the pattern.")

gvSession_df.shape

Concatenation successful.


(4884, 22)

**BEFORE PROCEEDING: Change the "directory" variable below to your own directory. Make sure to put the letter "r" at the start of the line, before the quotes, similar to below**

1. Next, we need to verify if the dimensions of the **combined gvSession dataframe**, which are the number of rows and columns, matches the total number of rows and columns of all **gvSession** files found in the current working directory.

2. Similar to above, we will loop through the directory to check if the file name of each file has the prefix "gvSession" contained within it, print out the number of rows and then sum up the total number of rows and columns to see if it tallies up with the current "gvSession" dataframe



In [5]:
# Directory where "gvSession" files are located, change this path within the quotes to your directory 
directory = directory_path

total_cols = 0
total_rows = 0

# Identify and loop through all "gvSession" files in the directory
for filename in os.listdir(directory):
    if filename.startswith('gvSession') and filename.endswith('.xlsx'):
        file_path = os.path.join(directory, filename)
        
        # Get the shape of each "gvSession" Excel file
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
        print(f"File: {filename}")
        num_rows = 0
        num_cols = 0
        
        for sheet_name in sheet_names:
            df = pd.read_excel(file_path, sheet_name=sheet_name)
            rows, cols = df.shape
            num_rows += rows
            num_cols += cols
            print(f"  - Sheet '{sheet_name}': Shape {df.shape}")
            
        for sheet_name in sheet_names:
            df = pd.read_excel(file_path, sheet_name=sheet_name)
            rows, cols = df.shape
            total_rows += rows
            total_cols += cols
        
        print(f"Total sum of shapes for '{filename}': {num_rows} rows, {num_cols} columns")
        print()  # Empty line for better readability between files

print(f"Total sum of rows and columns for all files: {total_rows} rows, 22 columns")

print()
#Compare with the shape of the combined gvSession dataframe 
print(f"Shape of the combined gvSession Dataframe: {gvSession_df.shape}")

File: gvSession (1).xlsx
  - Sheet 'Sheet': Shape (1000, 22)
Total sum of shapes for 'gvSession (1).xlsx': 1000 rows, 22 columns

File: gvSession (2).xlsx
  - Sheet 'Sheet': Shape (1000, 22)
Total sum of shapes for 'gvSession (2).xlsx': 1000 rows, 22 columns

File: gvSession (3).xlsx
  - Sheet 'Sheet': Shape (1000, 22)
Total sum of shapes for 'gvSession (3).xlsx': 1000 rows, 22 columns

File: gvSession (4).xlsx
  - Sheet 'Sheet': Shape (884, 22)
Total sum of shapes for 'gvSession (4).xlsx': 884 rows, 22 columns

File: gvSession.xlsx
  - Sheet 'Sheet': Shape (1000, 22)
Total sum of shapes for 'gvSession.xlsx': 1000 rows, 22 columns

Total sum of rows and columns for all files: 4884 rows, 22 columns

Shape of the combined gvSession Dataframe: (4884, 22)


## 2.0 Extract relevant columns from gvSession

From the combined "gvSession" dataframe, we will extract the following columns

- Dept
- Sch#
- Course Code
- Course Title
- Session Type
- Session Date
- Session Day
- S-Time
- E-Time
- Venue
- Sch Status

**Note that we only need values where the "Sch Status" Column indicates "Pending" or "Confirmed"**

In [6]:
filtered_columns_df = gvSession_df.loc[:,['Dept','Sch #','Course Code','Course Title','Session Type',
                              'Session Date','Session Day','S-Time','E-Time','Venue', 'Sch Status']]

#Filter the dataframe such that the "Sch Status" column indicates either "Pending" or "Confirmed"
filtered_columns_df = filtered_columns_df[filtered_columns_df['Sch Status'].isin(['Pending', 'Confirmed'])]
 
print("Rows with courses of all dates in gvSession:", filtered_columns_df.shape[1])

Rows with courses of all dates in gvSession: 11


### 2.1 Filter out rows such that 

1. For all other courses that are not conducted in SMU, filter out the data and upload to a separate sheet in the Facility Booking Template
2. Only courses that are conducted in SMU require venues to be booked, and these courses will be indicated as "SMUA01" in the "Venue" Column

In [7]:
physical_courses_df = filtered_columns_df[filtered_columns_df["Venue"] == "SMUA Room 1"]
other_courses_df = filtered_columns_df[filtered_columns_df['Venue'] != "SMUA Room 1"]

Check the number of rows in each dataframe

In [8]:
#Check the number of rows in both dataframes
print(f"Number of rows in other_courses_df: {other_courses_df.shape[0]}")
print(f"Number of rows in physical_courses_df: {physical_courses_df.shape[0]}")

Number of rows in other_courses_df: 1037
Number of rows in physical_courses_df: 2552


In [9]:
other_courses_df.sample(10)

,Dept,Sch #,Course Code,Course Title,Session Type,Session Date,Session Day,S-Time,E-Time,Venue,Sch Status
336,Finance & Technology,SCH24-Advanced Diploma-ACDPPM2-00002,ACDPPM2,Advanced Certificate in Data Protection Princi...,Morning #02,2024-12-05,Thursday,09:00 AM,01:00 PM,Online Class,Pending
4259,"Services, Operations and Business Improvement",SCH24-Multiple Certificate-PCMLM4-00001,PCMLM4,Professional Certificate in Machine Learning (...,Morning #01,2024-09-14,Saturday,09:00 AM,12:30 PM,Online Class,Pending
4366,"Human Capital, Management & Leadership",SCH24-Not Specified-PCCM2-00002,PCCM2,Professional Certificate in Coaching Module 2:...,Afternoon #05,2024-10-02,Wednesday,02:00 PM,06:00 PM,Online Class,Pending
780,Finance & Technology,SCH24-Not Specified-ACIETM1-00004,ACIETM1,Advanced Certificate in Innovative Educational...,NaN,2024-10-29,Tuesday,12:00 AM,11:59 PM,Asynchronous E-learning,Pending
1731,Finance & Technology,SCH24-Not Specified-ECCRMM4-00002,ECCRMM4,Executive Certificate in Corporate Relationshi...,NaN,2024-08-23,Friday,09:15 AM,10:45 AM,School of Accountancy Seminar Room 3-2,Pending
368,"Services, Operations and Business Improvement",SCH24-Multiple Certificate-PCPPM6-00003,PCPPM6,Professional Certificate in Python Programming...,Assessment #01,2024-08-31,Saturday,05:01 PM,06:01 PM,Online Class,Pending
1637,"Services, Operations and Business Improvement",SCH24-Graduate Diploma-FBS-00003,FBS,Foundations of Brand Storytelling,Assessment #01,2024-10-12,Saturday,05:01 PM,06:01 PM,Online Class,Pending
608,Finance & Technology,SCH24-Not Specified-CIPPE-00004,CIPPE,Certified Information Privacy Professional/ Eu...,Assessment #02,2024-11-20,Wednesday,02:00 PM,06:00 PM,Online Class,Pending
1187,Finance & Technology,SCH24-Not Specified-PAOP-00012,PAOP,PDPA - An Operational Perspective,Morning #01,2025-01-10,Friday,09:00 AM,12:30 PM,Online Class,Pending
4799,Finance & Technology,SCH24-Not Specified-EADA-00002,EADA,Ethics for Accountants in the Digital Age,Morning #02,2024-11-28,Thursday,09:00 AM,01:00 PM,To be confirmed,Pending


In [10]:
physical_courses_df.sample(10)

,Dept,Sch #,Course Code,Course Title,Session Type,Session Date,Session Day,S-Time,E-Time,Venue,Sch Status
1655,"Human Capital, Management & Leadership",SCH24-Not Specified-CAP-00001,CAP,Certified Analytical Professional,Afternoon #05,2024-11-18,Monday,02:00 PM,06:00 PM,SMUA Room 1,Pending
1719,Finance & Technology,SCH24-Not Specified-ACVAUTM1-00002,ACVAUTM1,Advanced Certificate in Visual Analytics Using...,Assessment #05,2024-09-20,Friday,05:01 PM,06:01 PM,SMUA Room 1,Pending
4216,Finance & Technology,SCH24-Advanced Diploma-ACSCITM2-00002,ACSCITM2,Advanced Certificate in Supply Chain Innovatio...,Morning #02,2024-12-31,Tuesday,09:00 AM,01:00 PM,SMUA Room 1,Pending
494,Finance & Technology,SCH24-Not Specified-CFTP2M5-00002,CFTP2M5,Chartered Fintech Professional (CFtP) Level 2 ...,Morning #01,2024-11-15,Friday,09:00 AM,12:30 PM,SMUA Room 1,Pending
894,Finance & Technology,SCH24-Not Specified-DAUPB-00026,DAUPB,Data Analytics Using Power BI,Afternoon #02,2024-09-03,Tuesday,01:30 PM,05:00 PM,SMUA Room 1,Pending
3136,Finance & Technology,SCH24-Not Specified-ACORMBM5-00001,ACORMBM5,Advanced Certificate in Online Reputation Mana...,Morning #02,2024-09-27,Friday,09:00 AM,01:00 PM,SMUA Room 1,Pending
2392,Finance & Technology,SCH24-Not Specified-ICCTGC-00003,ICCTGC,Innovation Culture Catalyst (ICC): The Game Ch...,NaN,2024-11-05,Tuesday,02:00 PM,06:00 PM,SMUA Room 1,Pending
2940,Business Management,SCH24-Advanced Diploma-GEM2-2024-1-00001,GEM2-2024-1,Growing Enterprises Module 2: People (English ...,Morning #01,2024-09-18,Wednesday,09:00 AM,12:00 PM,SMUA Room 1,Pending
3891,Finance & Technology,SCH24-Not Specified-FRICRW-00003,FRICRW,"Fund Raising, IPOs & Capital Restructuring Wor...",Morning #02,2024-11-12,Tuesday,09:00 AM,01:00 PM,SMUA Room 1,Pending
4332,"Services, Operations and Business Improvement",SCH24-Multiple Certificate-ACACADM1-00002,ACACADM1,Advanced Certificate in AWS Cloud Architecting...,Evening #01,2024-09-04,Wednesday,07:00 PM,10:30 PM,SMUA Room 1,Pending


Upload the Online Classes Dataframe into a separate sheet in the Facility Bookings Template File

In [11]:
#Specify the output file path, which will be FBS Template Excel File in this case, and the sheet name you want to put the data under
output_file_path = 'Facility+Booking+List.xlsx'
sheet_name = 'Courses not conducted in SMU'

# Open the existing workbook
workbook = load_workbook(output_file_path)

# Delete the sheet if it exists
if sheet_name in workbook.sheetnames:
    sheet = workbook[sheet_name]
    workbook.remove(sheet)
    workbook.save(output_file_path)

# Write the other_courses_df to the output file
with pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a') as writer:
    other_courses_df.to_excel(writer, sheet_name=sheet_name, index=False)

    # Access the workbook and the specific sheet
    workbook = writer.book
    sheet = writer.sheets[sheet_name]

    # Autofit column widths
    for column in sheet.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        sheet.column_dimensions[column_letter].width = adjusted_width

print("Data for all other courses not conducted not within SMU have been uploaded to Sheet: Courses not conducted in SMU")

Data for all other courses not conducted not within SMU have been uploaded to Sheet: Courses not conducted in SMU


## 3.0 Obtain Time From and Time To Values

For each course, obtain the "Time of Booking From " and "Time of Booking To" by separating the morning, afternoon and night courses 

In [12]:
#Rename the physical_courses_df dataframe
facility_booking_df = physical_courses_df

In [13]:
facility_booking_df.head()

,Dept,Sch #,Course Code,Course Title,Session Type,Session Date,Session Day,S-Time,E-Time,Venue,Sch Status
0,"Human Capital, Management & Leadership",SCH23-Not Specified-ACHMLM4-00001,ACHMLM4,Advanced Certificate in Healthcare Management ...,Assessment #02,2024-10-16,Wednesday,02:00 PM,06:00 PM,SMUA Room 1,Pending
1,Finance & Technology,SCH24-Not Specified-ACW3M3-00003,ACW3M3,Advanced Certificate in Web 3.0 Module 3: Leve...,Afternoon #05,2024-08-26,Monday,02:00 PM,06:00 PM,SMUA Room 1,Pending
2,"Human Capital, Management & Leadership",SCH24-Not Specified-ACSUSPTGWYW-00007,ACSUSPTGWYW,Advanced Communication Strategies: Using Strat...,Afternoon #05,2024-11-11,Monday,02:00 PM,06:00 PM,SMUA Room 1,Confirmed
3,Business Management,SCH23-Not Specified-CESGPBF-FCFD-00007,CESGPBF-FCFD,"Certificate in Environmental, Social, and Gove...",Morning #01,2024-11-13,Wednesday,09:00 AM,12:30 PM,SMUA Room 1,Pending
6,"Human Capital, Management & Leadership",SCH24-Not Specified-CHAP-00001,CHAP,Certified HR Analytical Professional,Morning #02,2024-10-07,Monday,09:00 AM,01:00 PM,SMUA Room 1,Pending


In [14]:
# Check if current DF has the right number of rows
print("Number of rows in current data frame: ", facility_booking_df.shape[0])

Number of rows in current data frame:  2552


Reformat the S-Time and E-Time columns into datetime format

Then we need to identify the Morning, Afternoon and Night Time Courses, and create separate dataframes for each type of session

1. For Morning courses, they start before 12pm
2. For Afternoon courses, they start at or after 12pm, no later than 7pm
3. For Night courses, they start at 7pm or late


In [15]:
# Creates a new DataFrame that is a separate object from the original
booking_time_df = facility_booking_df.copy()

# Create new "DT S-Time" and "DT E-Time" columns to define functions later on
booking_time_df['DT S-Time'] = pd.to_datetime(booking_time_df['S-Time'], format='%I:%M %p')
booking_time_df['DT E-Time'] = pd.to_datetime(booking_time_df['E-Time'], format='%I:%M %p')

# Night Courses that start at 7pm or later
night_df = booking_time_df[(booking_time_df["DT S-Time"] >= pd.to_datetime('19:00', format='%H:%M'))]

# Morning Courses that start before 12pm
morning_df = booking_time_df[booking_time_df["DT S-Time"] < pd.to_datetime('12:00', format='%H:%M')]

# Afternoon courses that start at or after 12pm, before 7pm
afternoon_df = booking_time_df[(booking_time_df["DT S-Time"] >= pd.to_datetime('12:00', format='%H:%M')) & 
                               (booking_time_df["DT S-Time"] < pd.to_datetime('19:00', format='%H:%M'))]

#check the shapes of the different dataframes
print("booking_time_df", booking_time_df.shape)
print("night_df", night_df.shape)
print("morning_df", morning_df.shape)
print("afternoon_df", afternoon_df.shape)
print("Total rows from the time period dataframes: ", (morning_df.shape[0] + afternoon_df.shape[0] + night_df.shape[0]))


booking_time_df (2552, 13)
night_df (136, 13)
morning_df (976, 13)
afternoon_df (1440, 13)
Total rows from the time period dataframes:  2552


In [16]:
#Check the columns in booking_time_df
booking_time_df.columns

Index(['Dept', 'Sch #', 'Course Code', 'Course Title', 'Session Type',
       'Session Date', 'Session Day', 'S-Time', 'E-Time', 'Venue',
       'Sch Status', 'DT S-Time', 'DT E-Time'],
      dtype='object')

### 3.1 Night Sessions

For all Weekday 7-10pm courses, The "Time Booked From" will always be **1 hour** before S-Time, and the "Time Booked To" will be the **same** as E-Time  

In [17]:
#Check the start times for the night courses
night_df['DT S-Time'].value_counts()

DT S-Time
1900-01-01 19:00:00    135
1900-01-01 20:01:00      1
Name: count, dtype: int64

In [18]:
# Function to add 1 hr before
def add_hour(time):
          return time - timedelta(minutes=60)

night_df["DT S-Time"] = night_df["DT S-Time"].apply(add_hour)

#Define a function to populate the "Time Booked From" and "Time Booked To" columns based on the "HH:MM AM/PM" format
def populate_time_booked_from(time):
        return time.strftime("%I:%M %p")

night_df['Time Booked From'] = night_df['DT S-Time'].apply(populate_time_booked_from)
night_df['Time Booked To'] = night_df['DT E-Time'].apply(populate_time_booked_from)

print("The number of night courses: ", night_df.shape[0])

The number of night courses:  136


In [19]:
night_df['Time Booked From'].value_counts()

Time Booked From
06:00 PM    135
07:01 PM      1
Name: count, dtype: int64

### 3.2 Morning & Afternoon Sessions

For Weekday + Weekend courses 
 - Time Booked To: Same as E-Time as reflected in gvSession
 - Time Booked From: 
    1. All morning sessions to start at **8am**
    2. Afternoon sessions to start **1hr before**  

Morning Courses

In [20]:
# Check the start times for the morning courses
morning_df['DT S-Time'].value_counts()  

DT S-Time
1900-01-01 09:00:00    958
1900-01-01 08:30:00     16
1900-01-01 09:15:00      2
Name: count, dtype: int64

In [21]:
# Populate "Time Booked From" to 8am

# S-Time is 8am for Morning Session
morning_df.loc[:,'Time Booked From'] = "08:00 AM"

#Check if all morning sessions start at 8am, by checking if there are any rows where the time booked from is not 8am
morning_df[morning_df['Time Booked From'] != "08:00 AM"]


,Dept,Sch #,Course Code,Course Title,Session Type,Session Date,Session Day,S-Time,E-Time,Venue,Sch Status,DT S-Time,DT E-Time,Time Booked From


In [22]:
#Check the updated timings in the "Time Booked From" column
morning_df['Time Booked From'].value_counts()

Time Booked From
08:00 AM    976
Name: count, dtype: int64

In [23]:
#Time Booked To will be the same as E-Time, as reflected in gvSession
morning_df["Time Booked To"] = morning_df["E-Time"]
 
morning_df.sample(10)

,Dept,Sch #,Course Code,Course Title,Session Type,Session Date,Session Day,S-Time,E-Time,Venue,Sch Status,DT S-Time,DT E-Time,Time Booked From,Time Booked To
1065,"Services, Operations and Business Improvement",SCH24-Not Specified-PCIMERM4-00002,PCIMERM4,Professional Certificate in Immersive Media an...,Morning #01,2024-08-31,Saturday,09:00 AM,12:30 PM,SMUA Room 1,Pending,1900-01-01 09:00:00,1900-01-01 12:30:00,08:00 AM,12:30 PM
2173,Finance & Technology,SCH24-Not Specified-IBCFB-00002,IBCFB,Investment Banking and Corporate Finance Bootcamp,Morning #02,2024-09-02,Monday,09:00 AM,01:00 PM,SMUA Room 1,Pending,1900-01-01 09:00:00,1900-01-01 13:00:00,08:00 AM,01:00 PM
4194,Finance & Technology,SCH24-Not Specified-CCA-00003,CCA,Corporate Credit Analysis,Morning #01,2024-10-09,Wednesday,09:00 AM,12:30 PM,SMUA Room 1,Pending,1900-01-01 09:00:00,1900-01-01 12:30:00,08:00 AM,12:30 PM
3249,Finance & Technology,SCH24-Not Specified-GCLTM8-00001,GCLTM8,Graduate Certificate in Law and Technology Mod...,Morning #02,2024-09-20,Friday,09:00 AM,01:00 PM,SMUA Room 1,Pending,1900-01-01 09:00:00,1900-01-01 13:00:00,08:00 AM,01:00 PM
245,"Services, Operations and Business Improvement",SCH24-Not Specified-DDRW-00002,DDRW,Data Driven Report Writing: Making Insightful ...,Morning #01,2024-11-15,Friday,09:00 AM,12:30 PM,SMUA Room 1,Pending,1900-01-01 09:00:00,1900-01-01 12:30:00,08:00 AM,12:30 PM
302,Finance & Technology,SCH24-Not Specified-ACMAIM4-00002,ACMAIM4,Advanced Certificate in Metaverse and Artifici...,Morning #01,2024-08-19,Monday,09:00 AM,12:30 PM,SMUA Room 1,Pending,1900-01-01 09:00:00,1900-01-01 12:30:00,08:00 AM,12:30 PM
1939,Finance & Technology,SCH24-Not Specified-ACORMBM2-00002,ACORMBM2,Advanced Certificate in Online Reputation Mana...,Morning #02,2024-09-13,Friday,09:00 AM,01:00 PM,SMUA Room 1,Pending,1900-01-01 09:00:00,1900-01-01 13:00:00,08:00 AM,01:00 PM
4063,Finance & Technology,SCH24-Advanced Diploma-ACLSCMM2-00001,ACLSCMM2,Advanced Certificate in Logistics and Supply C...,Morning #02,2024-12-31,Tuesday,09:00 AM,01:00 PM,SMUA Room 1,Confirmed,1900-01-01 09:00:00,1900-01-01 13:00:00,08:00 AM,01:00 PM
2400,"Services, Operations and Business Improvement",SCH23-Not Specified-ACCSMM2-00002,ACCSMM2,Advanced Certificate in Competitive Sales and ...,Morning #01,2024-11-16,Saturday,09:00 AM,12:30 PM,SMUA Room 1,Pending,1900-01-01 09:00:00,1900-01-01 12:30:00,08:00 AM,12:30 PM
4147,"Human Capital, Management & Leadership",SCH24-Not Specified-ACRSM2-00001,ACRSM2,Advanced Certificate in Future Ready Rewards S...,Morning #01,2024-09-20,Friday,09:00 AM,12:30 PM,SMUA Room 1,Pending,1900-01-01 09:00:00,1900-01-01 12:30:00,08:00 AM,12:30 PM


In [24]:
print("Number of morning classes: ", morning_df.shape[0])

Number of morning classes:  976


Afternoon Courses

In [25]:
afternoon_df['DT S-Time'].value_counts()

DT S-Time
1900-01-01 14:00:00    474
1900-01-01 13:30:00    452
1900-01-01 17:01:00    339
1900-01-01 16:01:00     51
1900-01-01 17:31:00     29
1900-01-01 13:45:00     22
1900-01-01 14:31:00     15
1900-01-01 15:01:00     13
1900-01-01 18:01:00     12
1900-01-01 12:31:00      9
1900-01-01 13:00:00      7
1900-01-01 18:00:00      5
1900-01-01 12:30:00      4
1900-01-01 17:00:00      3
1900-01-01 12:01:00      3
1900-01-01 16:31:00      2
Name: count, dtype: int64

In [26]:
# Populate "Time Book From" to 1 hr before "DT S-Time"

# Define a function to add 1 hr before
def add_hour(time):
          return time - timedelta(minutes=60)

afternoon_df["DT S-Time"] = afternoon_df["DT S-Time"].apply(add_hour)

def populate_time_booked_from(time):
      
        return time.strftime("%I:%M %p")


afternoon_df['Time Booked From'] = afternoon_df['DT S-Time'].apply(populate_time_booked_from)

In [27]:
#Check the updated values in the "Time Booked From" column
afternoon_df['Time Booked From'].value_counts()

Time Booked From
01:00 PM    474
12:30 PM    452
04:01 PM    339
03:01 PM     51
04:31 PM     29
12:45 PM     22
01:31 PM     15
02:01 PM     13
05:01 PM     12
11:31 AM      9
12:00 PM      7
05:00 PM      5
11:30 AM      4
04:00 PM      3
11:01 AM      3
03:31 PM      2
Name: count, dtype: int64

In [28]:
#Time booked to is the same as the original E-Time as reflected in gvSession
afternoon_df["Time Booked To"] = afternoon_df["E-Time"]
afternoon_df.sample(10)

,Dept,Sch #,Course Code,Course Title,Session Type,Session Date,Session Day,S-Time,E-Time,Venue,Sch Status,DT S-Time,DT E-Time,Time Booked From,Time Booked To
1336,Finance & Technology,SCH23-Advanced Diploma-ACLSCMM4-00001,ACLSCMM4,Advanced Certificate in Logistics and Supply C...,Assessment #02,2024-12-30,Monday,02:00 PM,06:00 PM,SMUA Room 1,Confirmed,1900-01-01 13:00:00,1900-01-01 18:00:00,01:00 PM,06:00 PM
4120,Finance & Technology,SCH24-Advanced Diploma-ACLSCMM2-00001,ACLSCMM2,Advanced Certificate in Logistics and Supply C...,NaN,2024-12-31,Tuesday,03:01 PM,06:01 PM,SMUA Room 1,Confirmed,1900-01-01 14:01:00,1900-01-01 18:01:00,02:01 PM,06:01 PM
2532,"Services, Operations and Business Improvement",SCH24-Not Specified-LSSGBC-00002,LSSGBC,Lean Six Sigma Green Belt Certification,NaN,2024-08-22,Thursday,01:45 PM,05:30 PM,SMUA Room 1,Pending,1900-01-01 12:45:00,1900-01-01 17:30:00,12:45 PM,05:30 PM
3675,Finance & Technology,SCH24-Not Specified-AFMCV-00002,AFMCV,Advanced Financial Modelling and Corporate Val...,Assessment #02,2024-10-24,Thursday,02:00 PM,06:00 PM,SMUA Room 1,Pending,1900-01-01 13:00:00,1900-01-01 18:00:00,01:00 PM,06:00 PM
3434,Finance & Technology,SCH24-Not Specified-GCDFM4-00003,GCDFM4,Graduate Certificate in Digital Finance Module...,Afternoon #05,2024-12-09,Monday,02:00 PM,06:00 PM,SMUA Room 1,Pending,1900-01-01 13:00:00,1900-01-01 18:00:00,01:00 PM,06:00 PM
2545,"Services, Operations and Business Improvement",SCH24-Not Specified-XHS-00003,XHS,Marketing with Xiao Hong Shu (XHS): Strategies...,Afternoon #02,2024-11-14,Thursday,01:30 PM,05:00 PM,SMUA Room 1,Pending,1900-01-01 12:30:00,1900-01-01 17:00:00,12:30 PM,05:00 PM
2797,Finance & Technology,SCH24-Advanced Diploma-ACSCITM1-00001,ACSCITM1,Advanced Certificate in Supply Chain Innovatio...,Assessment #02,2024-12-30,Monday,02:00 PM,06:00 PM,SMUA Room 1,Pending,1900-01-01 13:00:00,1900-01-01 18:00:00,01:00 PM,06:00 PM
233,"Services, Operations and Business Improvement",SCH24-Not Specified-CNW3-00002,CNW3,Certificate in Nutrition and Wellness: Diet an...,Afternoon #02,2024-11-09,Saturday,01:30 PM,05:00 PM,SMUA Room 1,Pending,1900-01-01 12:30:00,1900-01-01 17:00:00,12:30 PM,05:00 PM
3804,"Services, Operations and Business Improvement",SCH24-Not Specified-CNW1-00002,CNW1,Certificate in Nutrition and Wellness: Contemp...,Assessment #05,2024-10-19,Saturday,05:01 PM,06:01 PM,SMUA Room 1,Pending,1900-01-01 16:01:00,1900-01-01 18:01:00,04:01 PM,06:01 PM
3188,Finance & Technology,SCH24-Not Specified-TRMBC-00001,TRMBC,Trading and Risk Management of Bulk Commodities,Assessment #02,2024-09-25,Wednesday,02:00 PM,06:00 PM,SMUA Room 1,Pending,1900-01-01 13:00:00,1900-01-01 18:00:00,01:00 PM,06:00 PM


In [29]:
print("Number of afternoon classes: ", afternoon_df.shape[0])


Number of afternoon classes:  1440


In [30]:
#merge all time data frames together
booking_time_df = pd.concat([morning_df,afternoon_df,night_df], ignore_index=True)

In [31]:
# Quick check to see if number of rows still tally
print("Number of rows in current working dataframe: ", booking_time_df.shape[0])
print("Number of rows in previous dataframe:", facility_booking_df.shape[0])

Number of rows in current working dataframe:  2552
Number of rows in previous dataframe: 2552


In [32]:
#Check for any null values in the Time Booked From and Time Booked To columns
print("Time Booked From Col Count: ", booking_time_df["Time Booked From"].isnull().sum())
print("Time Booked To Col Count: ", booking_time_df["Time Booked To"].isnull().sum())

Time Booked From Col Count:  0
Time Booked To Col Count:  0


In [33]:
booking_time_df['Time Booked From'].value_counts()

Time Booked From
08:00 AM    976
01:00 PM    474
12:30 PM    452
04:01 PM    339
06:00 PM    135
03:01 PM     51
04:31 PM     29
12:45 PM     22
01:31 PM     15
02:01 PM     13
05:01 PM     12
11:31 AM      9
12:00 PM      7
05:00 PM      5
11:30 AM      4
04:00 PM      3
11:01 AM      3
03:31 PM      2
07:01 PM      1
Name: count, dtype: int64

### 3.3 Merging full day courses

Find rows with the Same Course Title and Session Date, 

1. The start time will be the earliest between the two 
2. The end time will be the latest of the row


In [34]:
merge_courses_df = booking_time_df.copy()
merge_courses_df.columns

Index(['Dept', 'Sch #', 'Course Code', 'Course Title', 'Session Type',
       'Session Date', 'Session Day', 'S-Time', 'E-Time', 'Venue',
       'Sch Status', 'DT S-Time', 'DT E-Time', 'Time Booked From',
       'Time Booked To'],
      dtype='object')

In [35]:
print(f"Number of rows before merging courses: {merge_courses_df.shape[0]}")

Number of rows before merging courses: 2552


In [36]:
#Convert the Start Time and End Time Columns to DateTime Objects
merge_courses_df['Start Timing'] = pd.to_datetime(merge_courses_df['Time Booked From'])
merge_courses_df['End Timing'] = pd.to_datetime(merge_courses_df['Time Booked To'])

merge_courses_df.head()

,Dept,Sch #,Course Code,Course Title,Session Type,Session Date,Session Day,S-Time,E-Time,Venue,Sch Status,DT S-Time,DT E-Time,Time Booked From,Time Booked To,Start Timing,End Timing
0,Business Management,SCH23-Not Specified-CESGPBF-FCFD-00007,CESGPBF-FCFD,"Certificate in Environmental, Social, and Gove...",Morning #01,2024-11-13,Wednesday,09:00 AM,12:30 PM,SMUA Room 1,Pending,1900-01-01 09:00:00,1900-01-01 12:30:00,08:00 AM,12:30 PM,2024-09-14 08:00:00,2024-09-14 12:30:00
1,"Human Capital, Management & Leadership",SCH24-Not Specified-CHAP-00001,CHAP,Certified HR Analytical Professional,Morning #02,2024-10-07,Monday,09:00 AM,01:00 PM,SMUA Room 1,Pending,1900-01-01 09:00:00,1900-01-01 13:00:00,08:00 AM,01:00 PM,2024-09-14 08:00:00,2024-09-14 13:00:00
2,"Human Capital, Management & Leadership",SCH24-Not Specified-ACABPMM2-00001,ACABPMM2,Advanced Certificate in Agile Business Practic...,Morning #02,2024-09-12,Thursday,09:00 AM,01:00 PM,SMUA Room 1,Pending,1900-01-01 09:00:00,1900-01-01 13:00:00,08:00 AM,01:00 PM,2024-09-14 08:00:00,2024-09-14 13:00:00
3,Finance & Technology,SCH24-Not Specified-ACMAIM5-00002,ACMAIM5,Advanced Certificate in Metaverse and Artifici...,Morning #01,2024-09-05,Thursday,09:00 AM,12:30 PM,SMUA Room 1,Pending,1900-01-01 09:00:00,1900-01-01 12:30:00,08:00 AM,12:30 PM,2024-09-14 08:00:00,2024-09-14 12:30:00
4,"Human Capital, Management & Leadership",SCH23-Not Specified-ACHMLM3-00001,ACHMLM3,Advanced Certificate in Healthcare Management ...,Morning #02,2024-09-24,Tuesday,09:00 AM,01:00 PM,SMUA Room 1,Pending,1900-01-01 09:00:00,1900-01-01 13:00:00,08:00 AM,01:00 PM,2024-09-14 08:00:00,2024-09-14 13:00:00


In [37]:
merge_courses_df = merge_courses_df.groupby(['Course Title', 'Session Date']).agg({'DT S-Time': 'first', 
                                                                           'DT E-Time': 'first',
                                                                            'Dept': 'first', 
                                                                            'Course Code': 'first',
                                                                                   'Sch #': 'first',
                                                                            'Session Type': 'first',
                                                                            'Session Day': 'first',
                                                                            'S-Time': 'first', 
                                                                            'E-Time': 'first',
                                                                            'Venue': 'first',
                                                                            'Start Timing': 'min',
                                                                            'End Timing': 'max'}).reset_index()

In [38]:
# Convert start_time and end_time back to time format
merge_courses_df['Time Booked From'] = merge_courses_df['Start Timing'].dt.strftime('%I:%M %p')
merge_courses_df['Time Booked To'] = merge_courses_df['End Timing'].dt.strftime('%I:%M %p')

In [39]:
print(f"Number of rows after merging courses: {merge_courses_df.shape[0]}")

Number of rows after merging courses: 1121


In [40]:
merge_courses_df

,Course Title,Session Date,DT S-Time,DT E-Time,Dept,Course Code,Sch #,Session Type,Session Day,S-Time,E-Time,Venue,Start Timing,End Timing,Time Booked From,Time Booked To
0,A Case Approach to Modelling Corporate Acquisi...,2024-10-13,1900-01-01 09:00:00,1900-01-01 13:00:00,Finance & Technology,ACAMCAB,SCH24-Not Specified-ACAMCAB-00002,Morning #02,Sunday,09:00 AM,01:00 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:00:00,08:00 AM,06:00 PM
1,A Case Approach to Modelling Corporate Acquisi...,2024-10-14,1900-01-01 09:00:00,1900-01-01 13:00:00,Finance & Technology,ACAMCAB,SCH24-Not Specified-ACAMCAB-00002,Morning #02,Monday,09:00 AM,01:00 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:01 PM
2,Adaptability In the Face of Disruptive Change:...,2024-09-11,1900-01-01 09:00:00,1900-01-01 12:30:00,Finance & Technology,AFDCP,SCH24-Not Specified-AFDCP-00001,Morning #01,Wednesday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:01 PM
3,Advanced Certificate in AWS Cloud Architecting...,2024-09-03,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",ACACADM1,SCH24-Multiple Certificate-ACACADM1-00002,Evening #01,Tuesday,07:00 PM,10:30 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:30:00,06:00 PM,10:30 PM
4,Advanced Certificate in AWS Cloud Architecting...,2024-09-04,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",ACACADM1,SCH24-Multiple Certificate-ACACADM1-00002,Evening #01,Wednesday,07:00 PM,10:30 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:30:00,06:00 PM,10:30 PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,Using A/B Tests and Econometric Methods for Da...,2024-11-14,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",UATEMDDDM,SCH24-Not Specified-UATEMDDDM-00002,Evening #01,Thursday,07:00 PM,10:30 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:30:00,06:00 PM,10:30 PM
1117,Using A/B Tests and Econometric Methods for Da...,2024-11-16,1900-01-01 09:00:00,1900-01-01 12:30:00,"Services, Operations and Business Improvement",UATEMDDDM,SCH24-Not Specified-UATEMDDDM-00002,Morning #01,Saturday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:01 PM
1118,Web3.0 as a Business Opportunity,2024-09-30,1900-01-01 09:00:00,1900-01-01 12:30:00,Finance & Technology,WBO,SCH24-Not Specified-WBO-00001,Morning #01,Monday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:01 PM
1119,Workplace Automation 101: Automation at Work w...,2024-10-14,1900-01-01 09:00:00,1900-01-01 13:00:00,Finance & Technology,WA1AWMPP,SCH24-Not Specified-WA1AWMPP-00002,Morning #02,Monday,09:00 AM,01:00 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:00:00,08:00 AM,06:00 PM


Verify merge booking

- From a list of unique Course Titles, look for a Course Title with two instances of the same day
- From the Facility Booking DF, check for the earliest start time and latest end time
- In the new Merge Booking DF, check if the two instances have been merged with the earliest start time and latest end time

In [41]:
facility_booking_df["Course Title"].value_counts() 

Course Title
Lean Six Sigma Green Belt Certification                                                                                                                        64
Lean Six Sigma Black Belt Certification                                                                                                                        31
Advanced Certificate in Visual Analytics Using Tableau Module 1: Unlocking Insights with Analytics                                                             25
Advanced Communication Strategies: Using Strategic Persuasion To Get What You Want                                                                             25
ChatGPT for Educators: Using it to your Advantage                                                                                                              25
                                                                                                                                                               ..
Professional Ce

In [42]:
facility_booking_df[(facility_booking_df["Course Title"] == "Borrowing Base Lending")]

,Dept,Sch #,Course Code,Course Title,Session Type,Session Date,Session Day,S-Time,E-Time,Venue,Sch Status
425,Finance & Technology,SCH24-Not Specified-BBL-00001,BBL,Borrowing Base Lending,Assessment #01,2024-10-25,Friday,05:01 PM,06:01 PM,SMUA Room 1,Pending
1049,Finance & Technology,SCH24-Not Specified-BBL-00001,BBL,Borrowing Base Lending,Morning #01,2024-10-25,Friday,09:00 AM,12:30 PM,SMUA Room 1,Pending
2816,Finance & Technology,SCH24-Not Specified-BBL-00001,BBL,Borrowing Base Lending,Afternoon #02,2024-10-25,Friday,01:30 PM,05:00 PM,SMUA Room 1,Pending


In [43]:
merge_courses_df[(merge_courses_df["Course Title"] == "Borrowing Base Lending")]

,Course Title,Session Date,DT S-Time,DT E-Time,Dept,Course Code,Sch #,Session Type,Session Day,S-Time,E-Time,Venue,Start Timing,End Timing,Time Booked From,Time Booked To
401,Borrowing Base Lending,2024-10-25,1900-01-01 09:00:00,1900-01-01 12:30:00,Finance & Technology,BBL,SCH24-Not Specified-BBL-00001,Morning #01,Friday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:01 PM


### 3.4 Weird Timings Rounded Down
In this section, we ensure that the time values in the "Time Booked From" and "Time Booked To" column will be perfect values

For example, 
1. If a course starts/ends at 06:01 PM, it is rounded down to 06:00 PM
2. If a course starts/ends at 05:31 PM, it is rounded down to 05:30 PM


Check if any of the values in the "Time Booked From" or "Time Booked To" column are weird

In [44]:

merge_courses_df['Time Booked To'].value_counts()

Time Booked To
06:01 PM    409
06:00 PM    267
05:00 PM    213
10:30 PM    100
10:45 PM     28
06:16 PM     18
07:01 PM     13
05:30 PM     12
01:31 PM      8
06:31 PM      8
05:01 PM      8
01:00 PM      6
06:45 PM      4
01:01 PM      4
12:00 PM      4
07:00 PM      3
10:45 AM      2
10:00 PM      2
09:00 PM      2
08:00 PM      2
04:00 PM      2
05:31 PM      2
12:30 PM      1
10:00 AM      1
09:01 PM      1
03:00 PM      1
Name: count, dtype: int64

In [45]:
merge_courses_df['Time Booked To'].value_counts()

Time Booked To
06:01 PM    409
06:00 PM    267
05:00 PM    213
10:30 PM    100
10:45 PM     28
06:16 PM     18
07:01 PM     13
05:30 PM     12
01:31 PM      8
06:31 PM      8
05:01 PM      8
01:00 PM      6
06:45 PM      4
01:01 PM      4
12:00 PM      4
07:00 PM      3
10:45 AM      2
10:00 PM      2
09:00 PM      2
08:00 PM      2
04:00 PM      2
05:31 PM      2
12:30 PM      1
10:00 AM      1
09:01 PM      1
03:00 PM      1
Name: count, dtype: int64

In [46]:
# Function to convert string time to datetime object
def convert_to_datetime(time_str):
    return datetime.strptime(time_str, '%I:%M %p')

# Function to round down to the nearest multiple of 5 minutes
def round_down_to_nearest_five(time):
    minutes = (time.minute // 5) * 5
    return time.replace(minute=minutes, second=0, microsecond=0)

# Convert the two columns to datetime objects
merge_courses_df['DT S-Time'] = merge_courses_df['Time Booked From'].apply(convert_to_datetime)
merge_courses_df['DT E-Time'] = merge_courses_df['Time Booked To'].apply(convert_to_datetime)

# Identify and isolate timings that don't end on a 5-minute mark
merge_courses_df['Original Start Time'] = merge_courses_df['Time Booked From']  # Keep a copy of the original start times
merge_courses_df['Original End Time'] = merge_courses_df['Time Booked To']  # Keep a copy of the original end times

# Round down start and end times to the nearest 5-minute mark
merge_courses_df['DT S-Time'] = merge_courses_df['DT S-Time'].apply(round_down_to_nearest_five)
merge_courses_df['DT E-Time'] = merge_courses_df['DT E-Time'].apply(round_down_to_nearest_five)

# Convert back to string format for consistency with original data
merge_courses_df['Time Booked From'] = merge_courses_df['DT S-Time'].apply(lambda x: x.strftime('%I:%M %p'))
merge_courses_df['Time Booked To'] = merge_courses_df['DT E-Time'].apply(lambda x: x.strftime('%I:%M %p'))

# Print the number of courses with start or end times that were rounded down
rounded_down_start = len(merge_courses_df[merge_courses_df['Original Start Time'] != merge_courses_df['Time Booked From']])
rounded_down_end = len(merge_courses_df[merge_courses_df['Original End Time'] != merge_courses_df['Time Booked To']])
print("Number of courses with start times rounded down:", rounded_down_start)
print("Number of courses with end times rounded down:", rounded_down_end)

# Check if any timings don't end on a 5-minute mark after rounding, and see if any are left
weird_timings_start = merge_courses_df[merge_courses_df['DT S-Time'].apply(lambda x: x.minute % 5) != 0]
weird_timings_end = merge_courses_df[merge_courses_df['DT E-Time'].apply(lambda x: x.minute % 5) != 0]
remaining_weird_timings_start = len(weird_timings_start)
remaining_weird_timings_end = len(weird_timings_end)
print("Are there any imperfect start timings left?", remaining_weird_timings_start > 0)
print("Are there any imperfect end timings left?", remaining_weird_timings_end > 0)


Number of courses with start times rounded down: 2
Number of courses with end times rounded down: 471
Are there any imperfect start timings left? False
Are there any imperfect end timings left? False


Check if there are any weird end timings left

In [47]:
merge_courses_df['Time Booked To'].value_counts()

Time Booked To
06:00 PM    676
05:00 PM    221
10:30 PM    100
10:45 PM     28
06:15 PM     18
07:00 PM     16
05:30 PM     14
01:00 PM     10
06:30 PM      8
01:30 PM      8
12:00 PM      4
06:45 PM      4
09:00 PM      3
10:00 PM      2
08:00 PM      2
04:00 PM      2
10:45 AM      2
10:00 AM      1
12:30 PM      1
03:00 PM      1
Name: count, dtype: int64

In [48]:
merge_courses_df['Time Booked From'].value_counts()

Time Booked From
08:00 AM    974
06:00 PM    135
05:00 PM      5
12:30 PM      4
04:00 PM      1
01:00 PM      1
02:00 PM      1
Name: count, dtype: int64

In [49]:
#drop the extra columns generated
merge_courses_df.drop(columns=['Original End Time', 'Original Start Time'], inplace=True)

In [50]:
#Verify changes made to "Time Booked To" column
merge_courses_df.sample(10)

,Course Title,Session Date,DT S-Time,DT E-Time,Dept,Course Code,Sch #,Session Type,Session Day,S-Time,E-Time,Venue,Start Timing,End Timing,Time Booked From,Time Booked To
1107,Uncovering Compliance and Operational Trends i...,2024-09-18,1900-01-01 08:00:00,1900-01-01 17:00:00,Finance & Technology,UCOTCI,SCH24-Not Specified-UCOTCI-00001,Morning #01,Wednesday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 17:00:00,08:00 AM,05:00 PM
529,Corporate Site Visits: Green Destination 1,2025-01-09,1900-01-01 08:00:00,1900-01-01 18:00:00,Business Management,SCTP-ACDSCSVGD1,SCH24-Not Specified-SCTP-ACDSCSVGD1-00001,Morning #01,Thursday,09:00 AM,12:00 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:00 PM
212,Advanced Certificate in Procurement Essentials...,2024-08-19,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,ACPEM1,SCH24-Advanced Diploma-ACPEM1-00001,Morning #02,Monday,09:00 AM,01:00 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:00:00,08:00 AM,06:00 PM
943,Mindfulness-based Strategic Awareness for Deci...,2024-12-31,1900-01-01 18:00:00,1900-01-01 21:00:00,Finance & Technology,MSADML,SCH24-Not Specified-MSADML-00003,None,Tuesday,07:00 PM,08:00 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 21:01:00,06:00 PM,09:00 PM
283,Advanced Certificate in Supply Chain Intellige...,2024-10-08,1900-01-01 08:00:00,1900-01-01 17:00:00,Finance & Technology,ACSCIM3,SCH24-Not Specified-ACSCIM3-00002,Morning #01,Tuesday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 17:00:00,08:00 AM,05:00 PM
318,Advanced Certificate in Visual Analytics Using...,2024-10-01,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,ACVAUTM2,SCH24-Not Specified-ACVAUTM2-00001,Morning #01,Tuesday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:00 PM
921,Loan Syndication: Executing a Successful Trans...,2024-08-20,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,LSEST,SCH24-Not Specified-LSEST-00001,Morning #02,Tuesday,09:00 AM,01:00 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:00 PM
581,Diet and Nutrition for Mental Wellness,2024-11-22,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",CNWDNMW,SCH24-Not Specified-CNWDNMW-00003,Evening #01,Friday,07:00 PM,10:30 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:30:00,06:00 PM,10:30 PM
197,Advanced Certificate in Organisation Developme...,2024-10-02,1900-01-01 08:00:00,1900-01-01 18:00:00,"Human Capital, Management & Leadership",ACODM1,SCH23-Not Specified-ACODM1-00002,Morning #02,Wednesday,09:00 AM,01:00 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:00 PM
874,Lean Six Sigma Black Belt Certification,2024-11-07,1900-01-01 18:00:00,1900-01-01 22:45:00,"Services, Operations and Business Improvement",LSSBBC,SCH24-Not Specified-LSSBBC-00001,None,Thursday,07:00 PM,10:45 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:45:00,06:00 PM,10:45 PM


## 4.0 Obtain Use Type Column

Use the "(Latest) SSG Masterlisting" Excel File as a Master List to match the IO Code to its respective Course Title/Course Code
- If IO Code present, Use Type is “Event” 
- If absent, Use Type is “AdHoc” 

In [51]:
ssg_master_listing_df.head()

,Column1,Course Title (F2F only),Course Title (Synchronous E-Learning only),Course Title (Classroom & Synchronous)\n[Blended Delivery Mode],TPG Course Code (F2F only),TPG Course Code (Synchronous E-learning only),TPG Course Code (Classroom & Synchronous)\n[Blended Delivery Mode],SCN Course Code for F2F,SCN Course Code (Synchronous E-learning),IBF for F2F,...,Healthcare,Hospitality & Lifestyle,Human Resource,Innovation & Business Improvement,Internationalisation,Leadership and Management,Supply Chain Management,SF Series Primary Economic Pillar,Column5,Column6
0,1.0,Agile for Successful Project Implementation,Agile for Successful Project Implementation (S...,NaN,TGS-2020501663,TGS-2020512848,NaN,CRS-N-0045079,CRS-N-0052991,NaN,...,,,,X,,,,Digital,NaN,NaN
1,2.0,Achieving Mastery and Success Through Growth M...,Achieving Mastery and Success Through Growth M...,NaN,TGS-2020001396,TGS-2020501599,NaN,NaN,CRS-N-0044320,NaN,...,,,,X,,X,,Care,NaN,NaN
2,3.0,Adopting DevOps,NaN,NaN,TGS-2020501660,NaN,NaN,CRS-N-0045076,NaN,NaN,...,,,,X,,,,Industry4.0,NaN,NaN
3,4.0,Advanced Communication Strategies: Using Strat...,Advanced Communication Strategies: Using Strat...,NaN,TGS-2020512939,TGS-2020501602,NaN,CRS-N-0053385,CRS-N-0044322,NaN,...,,,,,,X,,Care,NaN,NaN
4,5.0,Anti-Money Laundering and its Ecosystem,Anti-Money Laundering and its Ecosystem (Synch...,NaN,TGS-2020501697,TGS-2020513008,NaN,CRS-N-0045293,CRS-N-0053220,NaN,...,,,,,,,,Industry4.0,NaN,NaN


This section of the code is to standardize the naming format for the Course Title for merging later on

1. First, we extract only the "Course Title (F2F only)" and "IO Code" columns from the masterlisting file, into a dataframe
2. Then, define a function to standardize and clean up the naming format in this dataframe to merge later on 

In [52]:
# Take only necessary rows as we just need course title to reference and IO code for populating columns
master_df = ssg_master_listing_df[["Course Title (F2F only)","IO Code"]]

# Rename to Course Title so that it can be merged on this column
master_df.rename(columns = {'Course Title (F2F only)':'Course Title'}, inplace = True)

# Function to standardize course title naming format
def course_title_cleanup(title):
    
     if "(Classroom & Asynchronous)" in str(title):
        return title.split("(Classroom & Asynchronous)")[0]
     else:
        return title

# Apply the function to the Course Title Column
master_df['Course Title'] = master_df['Course Title'].apply(course_title_cleanup)

# Remove any possible trailing spaces 
master_df['Course Title'] = master_df['Course Title'].str.strip()

In [53]:
#Check the number of rows in the master_df
print("Number of rows in (Latest) SSG Approved_Master Listing: ", master_df.shape[0])

Number of rows in (Latest) SSG Approved_Master Listing:  1274


In [54]:
master_df.head()

,Course Title,IO Code
0,Agile for Successful Project Implementation,ZAC1D2104
1,Achieving Mastery and Success Through Growth M...,ZAC1C50055
2,Adopting DevOps,ZAC1D2020
3,Advanced Communication Strategies: Using Strat...,ZAC1C50045
4,Anti-Money Laundering and its Ecosystem,ZAC1D2030


In [55]:
#Merge the two dataframes
io_code_df = merge_courses_df.merge(master_df.drop_duplicates(subset=['Course Title']), how='left')

In [56]:
# Ensure number of rows are still the same upon mergin
print("Number of rows initially before the merge: ", merge_courses_df.shape[0])
print("Number of rows after adding IO Code: ", io_code_df.shape[0])

io_code_df.head()

Number of rows initially before the merge:  1121
Number of rows after adding IO Code:  1121


,Course Title,Session Date,DT S-Time,DT E-Time,Dept,Course Code,Sch #,Session Type,Session Day,S-Time,E-Time,Venue,Start Timing,End Timing,Time Booked From,Time Booked To,IO Code
0,A Case Approach to Modelling Corporate Acquisi...,2024-10-13,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,ACAMCAB,SCH24-Not Specified-ACAMCAB-00002,Morning #02,Sunday,09:00 AM,01:00 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:00:00,08:00 AM,06:00 PM,ZAC1D6029
1,A Case Approach to Modelling Corporate Acquisi...,2024-10-14,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,ACAMCAB,SCH24-Not Specified-ACAMCAB-00002,Morning #02,Monday,09:00 AM,01:00 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:00 PM,ZAC1D6029
2,Adaptability In the Face of Disruptive Change:...,2024-09-11,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,AFDCP,SCH24-Not Specified-AFDCP-00001,Morning #01,Wednesday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:00 PM,ZAC1D6326
3,Advanced Certificate in AWS Cloud Architecting...,2024-09-03,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",ACACADM1,SCH24-Multiple Certificate-ACACADM1-00002,Evening #01,Tuesday,07:00 PM,10:30 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:30:00,06:00 PM,10:30 PM,ZAC1A1225
4,Advanced Certificate in AWS Cloud Architecting...,2024-09-04,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",ACACADM1,SCH24-Multiple Certificate-ACACADM1-00002,Evening #01,Wednesday,07:00 PM,10:30 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:30:00,06:00 PM,10:30 PM,ZAC1A1225


A function will be defined to label the "Use Type" column as "Event" for courses with IO Code, and the rest of the courses will be labelled as "Ad-Hoc"

In [57]:
# Find the number of sessions without IO Code
io_code_df["IO Code"].isnull().sum()

270

In [58]:
# Sessions without IO Code will be labelled as "AdHoc"
io_code_df["IO Code"].fillna("AdHoc", inplace = True)

In [59]:
# For columns with IO Code, Use Type will be labelled as "Event"
def populate_use_type(io_code):
    
          if io_code != 'AdHoc':
            return "Event"
          else:
            return "AdHoc"

io_code_df["Use Type"] = io_code_df["IO Code"].apply(populate_use_type)

In [60]:
#Check the if the number of rows in the dataframe remains the same
print("Number of rows after adding IO Code: ", io_code_df.shape[0])

use_type_counts = io_code_df["Use Type"].value_counts()

#Conditional statements to check how many rows have "Event" or "AdHoc" in the "Use Type" column
if 'Event' in use_type_counts:
    print("Total Number of rows with 'Event' as Use-type: ", use_type_counts['Event'])
else:
    print("Total number of rows with 'Event' as Use-type: 0")

if 'AdHoc' in use_type_counts:
    print("Total number of rows with 'AdHoc' as Use-type: ", use_type_counts['AdHoc'])
else:
    print("Total Number of rows with 'AdHoc' as Use-type: 0")

Number of rows after adding IO Code:  1121
Total Number of rows with 'Event' as Use-type:  851
Total number of rows with 'AdHoc' as Use-type:  270


In [61]:
io_code_df[io_code_df["Use Type"] == "Event"]

,Course Title,Session Date,DT S-Time,DT E-Time,Dept,Course Code,Sch #,Session Type,Session Day,S-Time,E-Time,Venue,Start Timing,End Timing,Time Booked From,Time Booked To,IO Code,Use Type
0,A Case Approach to Modelling Corporate Acquisi...,2024-10-13,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,ACAMCAB,SCH24-Not Specified-ACAMCAB-00002,Morning #02,Sunday,09:00 AM,01:00 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:00:00,08:00 AM,06:00 PM,ZAC1D6029,Event
1,A Case Approach to Modelling Corporate Acquisi...,2024-10-14,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,ACAMCAB,SCH24-Not Specified-ACAMCAB-00002,Morning #02,Monday,09:00 AM,01:00 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:00 PM,ZAC1D6029,Event
2,Adaptability In the Face of Disruptive Change:...,2024-09-11,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,AFDCP,SCH24-Not Specified-AFDCP-00001,Morning #01,Wednesday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:00 PM,ZAC1D6326,Event
3,Advanced Certificate in AWS Cloud Architecting...,2024-09-03,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",ACACADM1,SCH24-Multiple Certificate-ACACADM1-00002,Evening #01,Tuesday,07:00 PM,10:30 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:30:00,06:00 PM,10:30 PM,ZAC1A1225,Event
4,Advanced Certificate in AWS Cloud Architecting...,2024-09-04,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",ACACADM1,SCH24-Multiple Certificate-ACACADM1-00002,Evening #01,Wednesday,07:00 PM,10:30 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:30:00,06:00 PM,10:30 PM,ZAC1A1225,Event
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107,Uncovering Compliance and Operational Trends i...,2024-09-18,1900-01-01 08:00:00,1900-01-01 17:00:00,Finance & Technology,UCOTCI,SCH24-Not Specified-UCOTCI-00001,Morning #01,Wednesday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 17:00:00,08:00 AM,05:00 PM,ZAC1D6270,Event
1108,Uncovering Compliance and Operational Trends i...,2024-09-19,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,UCOTCI,SCH24-Not Specified-UCOTCI-00001,Morning #01,Thursday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:00 PM,ZAC1D6270,Event
1118,Web3.0 as a Business Opportunity,2024-09-30,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,WBO,SCH24-Not Specified-WBO-00001,Morning #01,Monday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:00 PM,ZAC1D6327,Event
1119,Workplace Automation 101: Automation at Work w...,2024-10-14,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,WA1AWMPP,SCH24-Not Specified-WA1AWMPP-00002,Morning #02,Monday,09:00 AM,01:00 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:00:00,08:00 AM,06:00 PM,ZAC1D6187,Event


In [62]:
io_code_df[io_code_df["Use Type"] == "AdHoc"]

,Course Title,Session Date,DT S-Time,DT E-Time,Dept,Course Code,Sch #,Session Type,Session Day,S-Time,E-Time,Venue,Start Timing,End Timing,Time Booked From,Time Booked To,IO Code,Use Type
82,Advanced Certificate in Digital Supply Chain M...,2024-08-19,1900-01-01 08:00:00,1900-01-01 17:00:00,Finance & Technology,ACDSCM4,SCH24-Not Specified-ACDSCM4-00002,Morning #01,Monday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 17:00:00,08:00 AM,05:00 PM,AdHoc,AdHoc
83,Advanced Certificate in Digital Supply Chain M...,2024-08-20,1900-01-01 08:00:00,1900-01-01 17:00:00,Finance & Technology,ACDSCM4,SCH24-Not Specified-ACDSCM4-00002,Morning #01,Tuesday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 17:00:00,08:00 AM,05:00 PM,AdHoc,AdHoc
84,Advanced Certificate in Digital Supply Chain M...,2024-08-21,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,ACDSCM4,SCH24-Not Specified-ACDSCM4-00002,Morning #01,Wednesday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:00 PM,AdHoc,AdHoc
88,Advanced Certificate in Digital Supply Chain M...,2024-10-14,1900-01-01 08:00:00,1900-01-01 17:00:00,Finance & Technology,ACDSCM5,SCH24-Not Specified-ACDSCM5-00002,Morning #01,Monday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 17:00:00,08:00 AM,05:00 PM,AdHoc,AdHoc
89,Advanced Certificate in Digital Supply Chain M...,2024-10-15,1900-01-01 08:00:00,1900-01-01 17:00:00,Finance & Technology,ACDSCM5,SCH24-Not Specified-ACDSCM5-00002,Morning #01,Tuesday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 17:00:00,08:00 AM,05:00 PM,AdHoc,AdHoc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113,Unmanned Aerial Vehicle (UAV) Drone Piloting,2024-09-26,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",UAVDP,SCH23-Not Specified-UAVDP-00004,Evening #01,Thursday,07:00 PM,10:30 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:30:00,06:00 PM,10:30 PM,AdHoc,AdHoc
1114,Unmanned Aerial Vehicle (UAV) Drone Piloting,2024-09-27,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",UAVDP,SCH23-Not Specified-UAVDP-00004,Evening #01,Friday,07:00 PM,10:30 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:30:00,06:00 PM,10:30 PM,AdHoc,AdHoc
1115,Using A/B Tests and Econometric Methods for Da...,2024-11-13,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",UATEMDDDM,SCH24-Not Specified-UATEMDDDM-00002,Evening #01,Wednesday,07:00 PM,10:30 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:30:00,06:00 PM,10:30 PM,AdHoc,AdHoc
1116,Using A/B Tests and Econometric Methods for Da...,2024-11-14,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",UATEMDDDM,SCH24-Not Specified-UATEMDDDM-00002,Evening #01,Thursday,07:00 PM,10:30 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:30:00,06:00 PM,10:30 PM,AdHoc,AdHoc


## 5.0 Obtain Purpose Column

Standardized Format for the "Purpose" Column will be: | Course Code ~ Course Title |

In [63]:
purpose_df = io_code_df

In [64]:
def populate_purpose(row):
    
    return row['Course Code'] + " ~ " + row['Course Title']


purpose_df['Purpose'] = purpose_df.apply(populate_purpose,axis=1)

In [65]:
print("Number of rows after adding Purpose: ", purpose_df.shape[0])

Number of rows after adding Purpose:  1121


In [66]:
purpose_df.head()

,Course Title,Session Date,DT S-Time,DT E-Time,Dept,Course Code,Sch #,Session Type,Session Day,S-Time,E-Time,Venue,Start Timing,End Timing,Time Booked From,Time Booked To,IO Code,Use Type,Purpose
0,A Case Approach to Modelling Corporate Acquisi...,2024-10-13,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,ACAMCAB,SCH24-Not Specified-ACAMCAB-00002,Morning #02,Sunday,09:00 AM,01:00 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:00:00,08:00 AM,06:00 PM,ZAC1D6029,Event,ACAMCAB ~ A Case Approach to Modelling Corpora...
1,A Case Approach to Modelling Corporate Acquisi...,2024-10-14,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,ACAMCAB,SCH24-Not Specified-ACAMCAB-00002,Morning #02,Monday,09:00 AM,01:00 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:00 PM,ZAC1D6029,Event,ACAMCAB ~ A Case Approach to Modelling Corpora...
2,Adaptability In the Face of Disruptive Change:...,2024-09-11,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,AFDCP,SCH24-Not Specified-AFDCP-00001,Morning #01,Wednesday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:00 PM,ZAC1D6326,Event,AFDCP ~ Adaptability In the Face of Disruptive...
3,Advanced Certificate in AWS Cloud Architecting...,2024-09-03,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",ACACADM1,SCH24-Multiple Certificate-ACACADM1-00002,Evening #01,Tuesday,07:00 PM,10:30 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:30:00,06:00 PM,10:30 PM,ZAC1A1225,Event,ACACADM1 ~ Advanced Certificate in AWS Cloud A...
4,Advanced Certificate in AWS Cloud Architecting...,2024-09-04,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",ACACADM1,SCH24-Multiple Certificate-ACACADM1-00002,Evening #01,Wednesday,07:00 PM,10:30 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:30:00,06:00 PM,10:30 PM,ZAC1A1225,Event,ACACADM1 ~ Advanced Certificate in AWS Cloud A...


## 6.0 Obtain Event Code Column

The Event Code will be the IO Code for each course. However, for courses with no IO Code, the column will be populated with null values instead.

In [67]:
event_code_df = purpose_df

In [68]:
#Replace values in the IO Code column with null values if the value is "AdHoc
event_code_df["IO Code"].replace('AdHoc', np.nan, inplace=True)
print("Number of rows after adding Event Code: ", event_code_df.shape[0])


Number of rows after adding Event Code:  1121


In [69]:
event_code_df[event_code_df["Use Type"] == "AdHoc"]

,Course Title,Session Date,DT S-Time,DT E-Time,Dept,Course Code,Sch #,Session Type,Session Day,S-Time,E-Time,Venue,Start Timing,End Timing,Time Booked From,Time Booked To,IO Code,Use Type,Purpose
82,Advanced Certificate in Digital Supply Chain M...,2024-08-19,1900-01-01 08:00:00,1900-01-01 17:00:00,Finance & Technology,ACDSCM4,SCH24-Not Specified-ACDSCM4-00002,Morning #01,Monday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 17:00:00,08:00 AM,05:00 PM,NaN,AdHoc,ACDSCM4 ~ Advanced Certificate in Digital Supp...
83,Advanced Certificate in Digital Supply Chain M...,2024-08-20,1900-01-01 08:00:00,1900-01-01 17:00:00,Finance & Technology,ACDSCM4,SCH24-Not Specified-ACDSCM4-00002,Morning #01,Tuesday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 17:00:00,08:00 AM,05:00 PM,NaN,AdHoc,ACDSCM4 ~ Advanced Certificate in Digital Supp...
84,Advanced Certificate in Digital Supply Chain M...,2024-08-21,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,ACDSCM4,SCH24-Not Specified-ACDSCM4-00002,Morning #01,Wednesday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:00 PM,NaN,AdHoc,ACDSCM4 ~ Advanced Certificate in Digital Supp...
88,Advanced Certificate in Digital Supply Chain M...,2024-10-14,1900-01-01 08:00:00,1900-01-01 17:00:00,Finance & Technology,ACDSCM5,SCH24-Not Specified-ACDSCM5-00002,Morning #01,Monday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 17:00:00,08:00 AM,05:00 PM,NaN,AdHoc,ACDSCM5 ~ Advanced Certificate in Digital Supp...
89,Advanced Certificate in Digital Supply Chain M...,2024-10-15,1900-01-01 08:00:00,1900-01-01 17:00:00,Finance & Technology,ACDSCM5,SCH24-Not Specified-ACDSCM5-00002,Morning #01,Tuesday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 17:00:00,08:00 AM,05:00 PM,NaN,AdHoc,ACDSCM5 ~ Advanced Certificate in Digital Supp...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113,Unmanned Aerial Vehicle (UAV) Drone Piloting,2024-09-26,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",UAVDP,SCH23-Not Specified-UAVDP-00004,Evening #01,Thursday,07:00 PM,10:30 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:30:00,06:00 PM,10:30 PM,NaN,AdHoc,UAVDP ~ Unmanned Aerial Vehicle (UAV) Drone Pi...
1114,Unmanned Aerial Vehicle (UAV) Drone Piloting,2024-09-27,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",UAVDP,SCH23-Not Specified-UAVDP-00004,Evening #01,Friday,07:00 PM,10:30 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:30:00,06:00 PM,10:30 PM,NaN,AdHoc,UAVDP ~ Unmanned Aerial Vehicle (UAV) Drone Pi...
1115,Using A/B Tests and Econometric Methods for Da...,2024-11-13,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",UATEMDDDM,SCH24-Not Specified-UATEMDDDM-00002,Evening #01,Wednesday,07:00 PM,10:30 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:30:00,06:00 PM,10:30 PM,NaN,AdHoc,UATEMDDDM ~ Using A/B Tests and Econometric Me...
1116,Using A/B Tests and Econometric Methods for Da...,2024-11-14,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",UATEMDDDM,SCH24-Not Specified-UATEMDDDM-00002,Evening #01,Thursday,07:00 PM,10:30 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:30:00,06:00 PM,10:30 PM,NaN,AdHoc,UATEMDDDM ~ Using A/B Tests and Econometric Me...


In [70]:
event_code_df

,Course Title,Session Date,DT S-Time,DT E-Time,Dept,Course Code,Sch #,Session Type,Session Day,S-Time,E-Time,Venue,Start Timing,End Timing,Time Booked From,Time Booked To,IO Code,Use Type,Purpose
0,A Case Approach to Modelling Corporate Acquisi...,2024-10-13,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,ACAMCAB,SCH24-Not Specified-ACAMCAB-00002,Morning #02,Sunday,09:00 AM,01:00 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:00:00,08:00 AM,06:00 PM,ZAC1D6029,Event,ACAMCAB ~ A Case Approach to Modelling Corpora...
1,A Case Approach to Modelling Corporate Acquisi...,2024-10-14,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,ACAMCAB,SCH24-Not Specified-ACAMCAB-00002,Morning #02,Monday,09:00 AM,01:00 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:00 PM,ZAC1D6029,Event,ACAMCAB ~ A Case Approach to Modelling Corpora...
2,Adaptability In the Face of Disruptive Change:...,2024-09-11,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,AFDCP,SCH24-Not Specified-AFDCP-00001,Morning #01,Wednesday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:00 PM,ZAC1D6326,Event,AFDCP ~ Adaptability In the Face of Disruptive...
3,Advanced Certificate in AWS Cloud Architecting...,2024-09-03,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",ACACADM1,SCH24-Multiple Certificate-ACACADM1-00002,Evening #01,Tuesday,07:00 PM,10:30 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:30:00,06:00 PM,10:30 PM,ZAC1A1225,Event,ACACADM1 ~ Advanced Certificate in AWS Cloud A...
4,Advanced Certificate in AWS Cloud Architecting...,2024-09-04,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",ACACADM1,SCH24-Multiple Certificate-ACACADM1-00002,Evening #01,Wednesday,07:00 PM,10:30 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:30:00,06:00 PM,10:30 PM,ZAC1A1225,Event,ACACADM1 ~ Advanced Certificate in AWS Cloud A...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,Using A/B Tests and Econometric Methods for Da...,2024-11-14,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",UATEMDDDM,SCH24-Not Specified-UATEMDDDM-00002,Evening #01,Thursday,07:00 PM,10:30 PM,SMUA Room 1,2024-09-14 18:00:00,2024-09-14 22:30:00,06:00 PM,10:30 PM,NaN,AdHoc,UATEMDDDM ~ Using A/B Tests and Econometric Me...
1117,Using A/B Tests and Econometric Methods for Da...,2024-11-16,1900-01-01 08:00:00,1900-01-01 18:00:00,"Services, Operations and Business Improvement",UATEMDDDM,SCH24-Not Specified-UATEMDDDM-00002,Morning #01,Saturday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:00 PM,NaN,AdHoc,UATEMDDDM ~ Using A/B Tests and Econometric Me...
1118,Web3.0 as a Business Opportunity,2024-09-30,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,WBO,SCH24-Not Specified-WBO-00001,Morning #01,Monday,09:00 AM,12:30 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:01:00,08:00 AM,06:00 PM,ZAC1D6327,Event,WBO ~ Web3.0 as a Business Opportunity
1119,Workplace Automation 101: Automation at Work w...,2024-10-14,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,WA1AWMPP,SCH24-Not Specified-WA1AWMPP-00002,Morning #02,Monday,09:00 AM,01:00 PM,SMUA Room 1,2024-09-14 08:00:00,2024-09-14 18:00:00,08:00 AM,06:00 PM,ZAC1D6187,Event,WA1AWMPP ~ Workplace Automation 101: Automatio...


## 7.0 Obtain Facility Name, Building and Floor

Taking reference from FBS Report dataframe, if courses in the upcoming cycle occurred in the previous cycle as well, we will allocate these courses the same venue from the previous cycle 

Things to take note of:
- The Course Code may appear more than once in FBS Report, which means that a certain course may have run more than once in the same period.
- Only the latest venue will be extracted for each course

In [71]:
print(fbs_report_df.shape)
fbs_report_df.head()

(299, 25)


,Booking Reference Number,Building,Floor,Facility Type,Facility,Date of Transaction,Booking Date,Booking Start Time,Booking End Time,Duration of Booking,...,User's Cost Centre,Use Type,Course/Event Code,Purpose,BookingStatus,Rejected By,Previous Booking User,Reason,LastUpdatedDateTime,Count
0,BK-20240705-000531,School of Economics/School of Computing & Info...,Level 4,Seminar Room,SOE/SCIS2 Seminar Room 4-3,2024-07-05 09:19:00,2024-08-19,2024-07-09 18:00:00,2024-07-09 22:30:00,4.5,...,(C315) SMU Academy,Event,- / (ZAC1A1243) FY23P4-Advanced Certificate in...,ACVPTFERVP ~ Advanced Certificate in Virtual P...,Confirmed,,,,05/07/2024 09:19:39 AM,1
1,BK-20240705-000341,School of Economics/School of Computing & Info...,Level 3,Classroom,SOE/SCIS2 Classroom 3-1,2024-07-05 09:14:00,2024-08-29,2024-07-09 08:00:00,2024-07-09 17:00:00,9.0,...,(C315) SMU Academy,Event,- / (ZAC1D6339) FY23P3-Advanced Certificate in...,ACSCIM4 ~ Advanced Certificate in Supply Chain...,Confirmed,,,,05/07/2024 09:14:12 AM,1
2,CBK-20240703-001107,School of Economics/School of Computing & Info...,Level 2,Seminar Room,SOE/SCIS2 Seminar Room 2-8,2024-07-03 09:09:00,2024-08-26,2024-07-09 08:30:00,2024-07-09 18:00:00,9.5,...,(C315) SMU Academy,AdHoc,- / -,BWL (PUB),Confirmed,,,,08/07/2024 05:19:11 PM,1
3,BK-20240705-001270,Lee Kong Chian School of Business,Level 1,Seminar Room,LKCSB Seminar Room 1-2,2024-07-05 16:55:00,2024-08-23,2024-07-09 08:00:00,2024-07-09 18:00:00,10.0,...,(C315) SMU Academy,Event,- / (ZAC1D6273) FY23P3-Funding Startups,FUNDSU ~ Funding Startups,Confirmed,,,,05/07/2024 04:55:49 PM,1
4,CBK-20240703-001116,School of Economics/School of Computing & Info...,Level 2,Seminar Room,SOE/SCIS2 Seminar Room 2-6,2024-07-03 09:09:00,2024-08-27,2024-07-09 08:30:00,2024-07-09 18:00:00,9.5,...,(C315) SMU Academy,AdHoc,- / -,PWRW3M1,Withdrawn,,,,08/07/2024 05:14:16 PM,1


In this section, 

1. We filter the FBS Report based on confirmed venues only*.
2. Create a new column called "Facility Name" in the filtered FBS Report, which is a duplicate of the "Facility" Column, to ensure that naming convention between both the event_code_df and this dataframe is consistent
2. Next, we extract the columns necessary for the merge, where we perform a left merge based on the Purpose column in both dataframes
3. Before merging, we sort and group the FBS Report based on Purpose and Booking Date, to ensure we only keep the latest venues in this dataframe before merging. 
4. This will ensure that if a course code appears more than once in FBS Report, only the latest venue is extracted

*Note: The filtered FBS Report will exclude catering venues as well*

In [72]:
# Create a copy of event_code_df
event_code_copy = event_code_df.copy()

# Filter out the rows with the status "Confirmed"
confirmed_fbs_report = fbs_report_df[fbs_report_df["BookingStatus"] == "Confirmed"]

# Filter out the relevant columns from confirmed_fbs_report
confirmed_fbs_report = confirmed_fbs_report[['Building', 'Floor', 'Facility', 'Purpose', 'Booking Date']]

# Rename 'Facility' to 'Facility Name' in confirmed_fbs_report
confirmed_fbs_report['Facility Name'] = confirmed_fbs_report['Facility']

# Convert 'Booking Date' to datetime format
confirmed_fbs_report['Booking Date'] = pd.to_datetime(confirmed_fbs_report['Booking Date'])

# Filter away the catering venues as well
catering_venue_to_exclude = ['SOE/SCIS2 Catering Area 4A (Near to SR 4-2)', 
                             'SOE/SCIS2 Catering Area 4B (Near to SR 4-2)', 
                             'SOE/SCIS2 Catering Area 4C (Near to SR 4-4)']
confirmed_fbs_report = confirmed_fbs_report[~confirmed_fbs_report['Facility Name'].isin(catering_venue_to_exclude)]

# Sort confirmed_fbs_report by 'Purpose' and 'Booking Date' in descending order
confirmed_fbs_report = confirmed_fbs_report.sort_values(by=['Purpose', 'Booking Date'], ascending=False)

# Keep only the latest venue (based on booking date) for each course
latest_confirmed_fbs_report = confirmed_fbs_report.groupby('Purpose').first().reset_index()

# Perform left merge on 'Purpose' column
merged_df = pd.merge(event_code_copy, latest_confirmed_fbs_report, on='Purpose', how='left')

merged_df


,Course Title,Session Date,DT S-Time,DT E-Time,Dept,Course Code,Sch #,Session Type,Session Day,S-Time,...,Time Booked From,Time Booked To,IO Code,Use Type,Purpose,Building,Floor,Facility,Booking Date,Facility Name
0,A Case Approach to Modelling Corporate Acquisi...,2024-10-13,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,ACAMCAB,SCH24-Not Specified-ACAMCAB-00002,Morning #02,Sunday,09:00 AM,...,08:00 AM,06:00 PM,ZAC1D6029,Event,ACAMCAB ~ A Case Approach to Modelling Corpora...,NaN,NaN,NaN,NaT,NaN
1,A Case Approach to Modelling Corporate Acquisi...,2024-10-14,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,ACAMCAB,SCH24-Not Specified-ACAMCAB-00002,Morning #02,Monday,09:00 AM,...,08:00 AM,06:00 PM,ZAC1D6029,Event,ACAMCAB ~ A Case Approach to Modelling Corpora...,NaN,NaN,NaN,NaT,NaN
2,Adaptability In the Face of Disruptive Change:...,2024-09-11,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,AFDCP,SCH24-Not Specified-AFDCP-00001,Morning #01,Wednesday,09:00 AM,...,08:00 AM,06:00 PM,ZAC1D6326,Event,AFDCP ~ Adaptability In the Face of Disruptive...,NaN,NaN,NaN,NaT,NaN
3,Advanced Certificate in AWS Cloud Architecting...,2024-09-03,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",ACACADM1,SCH24-Multiple Certificate-ACACADM1-00002,Evening #01,Tuesday,07:00 PM,...,06:00 PM,10:30 PM,ZAC1A1225,Event,ACACADM1 ~ Advanced Certificate in AWS Cloud A...,NaN,NaN,NaN,NaT,NaN
4,Advanced Certificate in AWS Cloud Architecting...,2024-09-04,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",ACACADM1,SCH24-Multiple Certificate-ACACADM1-00002,Evening #01,Wednesday,07:00 PM,...,06:00 PM,10:30 PM,ZAC1A1225,Event,ACACADM1 ~ Advanced Certificate in AWS Cloud A...,NaN,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,Using A/B Tests and Econometric Methods for Da...,2024-11-14,1900-01-01 18:00:00,1900-01-01 22:30:00,"Services, Operations and Business Improvement",UATEMDDDM,SCH24-Not Specified-UATEMDDDM-00002,Evening #01,Thursday,07:00 PM,...,06:00 PM,10:30 PM,NaN,AdHoc,UATEMDDDM ~ Using A/B Tests and Econometric Me...,NaN,NaN,NaN,NaT,NaN
1117,Using A/B Tests and Econometric Methods for Da...,2024-11-16,1900-01-01 08:00:00,1900-01-01 18:00:00,"Services, Operations and Business Improvement",UATEMDDDM,SCH24-Not Specified-UATEMDDDM-00002,Morning #01,Saturday,09:00 AM,...,08:00 AM,06:00 PM,NaN,AdHoc,UATEMDDDM ~ Using A/B Tests and Econometric Me...,NaN,NaN,NaN,NaT,NaN
1118,Web3.0 as a Business Opportunity,2024-09-30,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,WBO,SCH24-Not Specified-WBO-00001,Morning #01,Monday,09:00 AM,...,08:00 AM,06:00 PM,ZAC1D6327,Event,WBO ~ Web3.0 as a Business Opportunity,NaN,NaN,NaN,NaT,NaN
1119,Workplace Automation 101: Automation at Work w...,2024-10-14,1900-01-01 08:00:00,1900-01-01 18:00:00,Finance & Technology,WA1AWMPP,SCH24-Not Specified-WA1AWMPP-00002,Morning #02,Monday,09:00 AM,...,08:00 AM,06:00 PM,ZAC1D6187,Event,WA1AWMPP ~ Workplace Automation 101: Automatio...,NaN,NaN,NaN,NaT,NaN


In [73]:
# Example: Count null values in Facility Name column after left merge
merged_df['Facility Name'].isnull().sum()

1014

## 8.0  Clean Up Data Frame

1. Remove all unnecessary columns, and keep only the columns as per what is contained in the Facility Booking List Template File
2. Change Session Date Format from YYYY-MM-DD to dd-MMM-yyyy to match the format required

In [74]:
# Remove Unnecessary Columns 
cleaned_df = merged_df[["Facility Name","Building","Floor","Session Date","Time Booked From","Time Booked To",
                     "Use Type","Purpose","IO Code","Course Code","Sch #"]]

cleaned_df.rename(columns = {'Session Date':'Date of Booking',
                           'Time Booked From':'Time of Booking From',
                           'Time Booked To':'Time of Booking To',
                           'IO Code':'Event Code'}, inplace = True)

In [75]:
# Change the Format for the "Date of Booking" Column from YYYY-MM-DD to dd-MMM-yyyy
cleaned_df['Date of Booking'] = cleaned_df['Date of Booking'].dt.strftime('%d-%b-%Y')

In [76]:
#Check the number of rows in the cleaned dataframe
print("Number of current rows: ", cleaned_df.shape[0])

Number of current rows:  1121


In [77]:
#Check the number of courses for which venues are required, but have not been assigned
print(cleaned_df['Facility Name'].isnull().sum())
cleaned_df

1014


,Facility Name,Building,Floor,Date of Booking,Time of Booking From,Time of Booking To,Use Type,Purpose,Event Code,Course Code,Sch #
0,NaN,NaN,NaN,13-Oct-2024,08:00 AM,06:00 PM,Event,ACAMCAB ~ A Case Approach to Modelling Corpora...,ZAC1D6029,ACAMCAB,SCH24-Not Specified-ACAMCAB-00002
1,NaN,NaN,NaN,14-Oct-2024,08:00 AM,06:00 PM,Event,ACAMCAB ~ A Case Approach to Modelling Corpora...,ZAC1D6029,ACAMCAB,SCH24-Not Specified-ACAMCAB-00002
2,NaN,NaN,NaN,11-Sep-2024,08:00 AM,06:00 PM,Event,AFDCP ~ Adaptability In the Face of Disruptive...,ZAC1D6326,AFDCP,SCH24-Not Specified-AFDCP-00001
3,NaN,NaN,NaN,03-Sep-2024,06:00 PM,10:30 PM,Event,ACACADM1 ~ Advanced Certificate in AWS Cloud A...,ZAC1A1225,ACACADM1,SCH24-Multiple Certificate-ACACADM1-00002
4,NaN,NaN,NaN,04-Sep-2024,06:00 PM,10:30 PM,Event,ACACADM1 ~ Advanced Certificate in AWS Cloud A...,ZAC1A1225,ACACADM1,SCH24-Multiple Certificate-ACACADM1-00002
...,...,...,...,...,...,...,...,...,...,...,...
1116,NaN,NaN,NaN,14-Nov-2024,06:00 PM,10:30 PM,AdHoc,UATEMDDDM ~ Using A/B Tests and Econometric Me...,NaN,UATEMDDDM,SCH24-Not Specified-UATEMDDDM-00002
1117,NaN,NaN,NaN,16-Nov-2024,08:00 AM,06:00 PM,AdHoc,UATEMDDDM ~ Using A/B Tests and Econometric Me...,NaN,UATEMDDDM,SCH24-Not Specified-UATEMDDDM-00002
1118,NaN,NaN,NaN,30-Sep-2024,08:00 AM,06:00 PM,Event,WBO ~ Web3.0 as a Business Opportunity,ZAC1D6327,WBO,SCH24-Not Specified-WBO-00001
1119,NaN,NaN,NaN,14-Oct-2024,08:00 AM,06:00 PM,Event,WA1AWMPP ~ Workplace Automation 101: Automatio...,ZAC1D6187,WA1AWMPP,SCH24-Not Specified-WA1AWMPP-00002


## 9.0 Additional Checks to Perform

1. For courses that run for more than 1 day, assign the same venue
2. Check for venue clashes and see if 2 or more courses have the same venue, and correct accordingly
3. Book catering for the respective venues
4. Generate the "Venue Preference" Column
5. If the venue assigned to a course matches what was indicated in SMUA Venue Matters: Indicate "match", if not indicate as "Not Match"
6. If unable to find the course code in SMUA venue matters: Indicate "Preference not indicated"
7. Generate the "No.of Course Days column
8. If course code cannot be found in FBS Report, indicate as "Assignment of Venue required"
9. Ensure that all the facility names follow the naming convention suitable for FBS, using Facility Names File to look up the correct naming  

In [78]:
#Check for any missing values in the Facility Name Column 
venue_is_nan_rows = cleaned_df[cleaned_df["Facility Name"].isnull()]  

print(f"Number of missing 'Facility Name' values in the dataframe: {(cleaned_df['Facility Name'].isnull().sum())}") 

Number of missing 'Facility Name' values in the dataframe: 1014


### 9.1 Assign the same venue for courses that run for more than 1 day

1. This section will start by sorting the cleaned_df by "Purpose" and "Date of Booking", to ensure that courses are grouped together and sorted by the date.
2. To ensure that multiple-day courses have the same venue assigned throughout its entire run duration, the code will extract the venue details for the first day of the course and assign that venue to all the days that the course runs for
3. It will also account for the fact that some of these courses may not run consecutively, and may have a 1-day break in between as well.

In [79]:
# Define a function to parse the date-string into a datetime object if not already in that format
def parse_date(date_str):
    return datetime.strptime(date_str, '%d-%b-%Y')


# Assuming 'cleaned_df' is your input DataFrame
cleaned_df['Date of Booking'] = cleaned_df['Date of Booking'].apply(parse_date)
cleaned_df.sort_values(by=['Purpose', 'Date of Booking'], inplace=True)

# Function to check for consecutive dates with a possible one-day break
def has_consecutive_dates_with_one_day_break(dates):
    dates.sort()
    for i in range(len(dates) - 1):
        gap = (dates[i + 1] - dates[i]).days
        if gap != 1 and gap != 2:  # Allow for consecutive days or a one-day break
            return False
    return True

# Function to assign the same venue for consecutive courses
def assign_same_venue(df):
    for purpose, group in df.groupby('Purpose'):
        dates = group['Date of Booking'].tolist()
        if len(dates) > 1 and has_consecutive_dates_with_one_day_break(dates):
            first_facility_name = group.iloc[0]['Facility Name']
            first_building = group.iloc[0]['Building']
            first_floor = group.iloc[0]['Floor']
            
            # Update all rows for this purpose with the first venue details
            df.loc[df['Purpose'] == purpose, 'Facility Name'] = first_facility_name
            df.loc[df['Purpose'] == purpose, 'Building'] = first_building
            df.loc[df['Purpose'] == purpose, 'Floor'] = first_floor
    return df

# Apply the function to the DataFrame
same_venues_df = assign_same_venue(cleaned_df)

# Convert the 'Date of Booking' column back to string format for consistency
same_venues_df['Date of Booking'] = same_venues_df['Date of Booking'].dt.strftime('%d-%b-%Y')
same_venues_df.reset_index(drop=True, inplace=True)

We will now double check if all courses that run for multiple days have been assigned the same venue throughout its entire run duration

In [80]:
# Define a function to check if there are still any courses with different venues during its entire run duration
def check_venue_conflicts(group):
    '''
    This function checks, for courses that run for multiple days, if there still are different venues during the entire 
    run duration of the course. 
    '''
    unique_venues = group['Facility Name'].dropna().unique()
    if len(unique_venues) > 1:
        return True
    return False

#Group the dataframe by "Purpose" and filter out the rows with venue conflicts
conflict_purposes = same_venues_df.groupby('Purpose').filter(check_venue_conflicts)

if not conflict_purposes.empty:
    print("The following courses still have different venues during its run duration:")
    print(conflict_purposes[['Purpose', 'Date of Booking', 'Facility Name']])
else:
    print("All courses that run for multiple days have been assigned the same venue for the entire duration.")

All courses that run for multiple days have been assigned the same venue for the entire duration.


Run this code if the above code cell indicates that there are still multiple_day courses which have different venues during its run duration 

1. (SHORTCUT) Highlight the whole code chunk, and click CTRL + FORWARD SLASH to uncomment out the code
2. Run the code 

In [81]:
# # Function to rectify venue conflicts
# def rectify_venue_conflicts(df):
#     for purpose, group in df.groupby('Purpose'):
#         unique_venues = group['Facility Name'].dropna().unique()
#         if len(unique_venues) > 1:
#             first_venue = group['Facility Name'].dropna().iloc[0]
#             df.loc[df['Purpose'] == purpose, 'Facility Name'] = first_venue
#     return df

# # Convert 'Date of Booking' to datetime
# same_venues_df['Date of Booking'] = same_venues_df['Date of Booking'].apply(parse_date)

# # Check and rectify conflicts
# conflict_purposes = same_venues_df.groupby('Purpose').filter(check_venue_conflicts)

# if not conflict_purposes.empty:
#     print("The following courses still have venue conflicts before rectification:")
#     print(conflict_purposes[['Purpose', 'Date of Booking', 'Facility Name']])

#     # Rectify conflicts
#     same_venues_df = rectify_venue_conflicts(same_venues_df)

#     # Check for conflicts again after rectification
#     conflict_purposes_after = same_venues_df.groupby('Purpose').filter(check_venue_conflicts)

#     if not conflict_purposes_after.empty:
#         print("The following courses still have venue conflicts after rectification:")
#         print(conflict_purposes_after[['Purpose', 'Date of Booking', 'Facility Name']])
#     else:
#         print("No courses have venue conflicts after rectification.")
# else:
#     print("No courses have venue conflicts.")

# # Convert the 'Date of Booking' column back to string format for consistency
# same_venues_df['Date of Booking'] = same_venues_df['Date of Booking'].dt.strftime('%d-%b-%Y')

# same_venues_df.reset_index(drop=True, inplace=True)

### 9.2 Identifying Any Venue Clashes Between 2 or more Courses
Ensure that no two courses or more that are running on the same date have the same venues booked. 

**Code logic:**

For two courses with clashing venues, eg. course A, B

1. When identified, check if either course has the next latest venue. 
2. if found for both course A and B/only one course, leave the venue as it is for course A, and assign the next latest venue for course B
3. If not found for either course, leave the venue as it is for one course, and indicate as "Assignment of venue required" for the other

For more than 2 courses, eg. 3 courses with clashing venues, eg course A, B, C

1. When identified, check if the courses have the next latest venue available. 
2. If only one course, course A for example, has the next latest venue, then leave the original venue as it is for course B, and indicate "Assignment of venue required" for course C  
3. If 2 courses, eg. A and B have the next latest venue, leave the original venue as it is for course C, and change the venues accordingly for A and B
4. If all three courses have next latest venue, leave the original venue as it is for course A, and change the venues as accordingly for course B, C

Overall, ensure that if venue clashes are identified, the venue is changed for all days of the course

In [82]:
# Dbl check for venues being booked on the same day for two seperate courses
dbl_booking_check = same_venues_df[same_venues_df.duplicated
                             (subset=['Facility Name','Date of Booking','Time of Booking From'])]

dbl_booking_check.dropna(subset=['Building'], inplace=True)

In [83]:
print("Number of rows affected by double booking: ", dbl_booking_check.shape[0])

Number of rows affected by double booking:  0


In [84]:
def find_next_latest_venue(facility_name, booking_date):
    # Making use of the confirmed_fbs_report defined in section 6.0,
    # Filter  for the given facility name and dates after booking_date
    filtered_df = confirmed_fbs_report_df[(confirmed_fbs_report_df['Facility Name'] == facility_name) & 
                                          (confirmed_fbs_report_df['Booking Date'] > booking_date)]
    # Sort by Booking Date to get the next latest venue
    if not filtered_df.empty:
        next_venue = filtered_df.sort_values('Booking Date').iloc[0]
        return next_venue['Facility Name'], next_venue['Booking Date']
    
    return None, None

def resolve_venue_clashes(df):
    def replace_venue(row, next_venue):
        row['Facility Name'] = next_venue[0]
        row['Booking Date'] = next_venue[1]
        return row

    for purpose, group in df.groupby('Purpose'):
        clashing_courses = group[group.duplicated(subset=['Facility Name', 'Date of Booking', 'Time of Booking From'], keep=False)]
        if not clashing_courses.empty:
            venues_available = []
            for index, row in clashing_courses.iterrows():
                next_venue = find_next_latest_venue(row['Facility Name'], row['Date of Booking'])
                if next_venue[0] is not None:
                    venues_available.append(next_venue)
            
            # Resolve clashes based on number of available venues
            num_courses = len(clashing_courses)
            if len(venues_available) >= num_courses - 1:
                # Assign the next available venues
                for idx, (index, row) in enumerate(clashing_courses.iterrows()):
                    if idx < num_courses - 1:
                        df.loc[index] = replace_venue(row, venues_available[idx])
            else:
                # If not enough venues, leave one and mark others as "Assignment of venue required"
                for idx, (index, row) in enumerate(clashing_courses.iterrows()):
                    if idx >= len(venues_available):
                        df.loc[index, 'Facility Name'] = 'Assignment of venue required'
    return df

# Apply the function to the DataFrame
clash_resolved_df = resolve_venue_clashes(same_venues_df)

In [85]:
#Check if there is still double bookings
dbl_booking_check = clash_resolved_df[clash_resolved_df.duplicated
                             (subset=['Facility Name','Date of Booking','Time of Booking From'])]

dbl_booking_check.dropna(subset=['Building'], inplace=True)


no_venues_df = dbl_booking_check[dbl_booking_check['Facility Name'] == 'Assignment of venue required']
blank_df = dbl_booking_check[dbl_booking_check['Facility Name'] == '']

print("Number of rows affected by double booking: ", dbl_booking_check.shape[0] - no_venues_df.shape[0] - blank_df.shape[0] )

print(f"Total Number of rows: {clash_resolved_df.shape[0]}")

Number of rows affected by double booking:  0
Total Number of rows: 1121


### 9.3 Booking of Catering Areas

1. Venues that require an additional catering venue are: SOE/SCIS2 CTE Scape, SOE/SCIS2 Seminar Room 5-1, and SOE/SCIS2 Seminar Room 5-2           
2. For these venues, either SOE/SCIS2 Catering Area 4A or SOE/SCIS2 Catering Area 4B will be booked
3. The row indicating the booking for catering venue will appear directly below the row of the course for which the above mentioned venues are booked 
4. Night courses do not require catering

In [86]:
clash_resolved_df.shape

clash_resolved_df

,Facility Name,Building,Floor,Date of Booking,Time of Booking From,Time of Booking To,Use Type,Purpose,Event Code,Course Code,Sch #
0,NaN,NaN,NaN,11-Sep-2024,08:00 AM,06:00 PM,Event,ACABPMM2 ~ Advanced Certificate in Agile Busin...,ZAC1C50073,ACABPMM2,SCH24-Not Specified-ACABPMM2-00001
1,NaN,NaN,NaN,12-Sep-2024,08:00 AM,06:00 PM,Event,ACABPMM2 ~ Advanced Certificate in Agile Busin...,ZAC1C50073,ACABPMM2,SCH24-Not Specified-ACABPMM2-00001
2,NaN,NaN,NaN,19-Aug-2024,08:00 AM,06:00 PM,Event,ACABPMM3 ~ Advanced Certificate in Agile Busin...,ZAC1C50074,ACABPMM3,SCH24-Not Specified-ACABPMM3-00001
3,NaN,NaN,NaN,20-Aug-2024,08:00 AM,06:00 PM,Event,ACABPMM3 ~ Advanced Certificate in Agile Busin...,ZAC1C50074,ACABPMM3,SCH24-Not Specified-ACABPMM3-00001
4,NaN,NaN,NaN,21-Aug-2024,08:00 AM,06:00 PM,Event,ACABPMM3 ~ Advanced Certificate in Agile Busin...,ZAC1C50074,ACABPMM3,SCH24-Not Specified-ACABPMM3-00001
...,...,...,...,...,...,...,...,...,...,...,...
1116,NaN,NaN,NaN,14-Oct-2024,08:00 AM,06:00 PM,Event,WA1AWMPP ~ Workplace Automation 101: Automatio...,ZAC1D6187,WA1AWMPP,SCH24-Not Specified-WA1AWMPP-00002
1117,NaN,NaN,NaN,15-Oct-2024,08:00 AM,06:00 PM,Event,WA1AWMPP ~ Workplace Automation 101: Automatio...,ZAC1D6187,WA1AWMPP,SCH24-Not Specified-WA1AWMPP-00002
1118,NaN,NaN,NaN,30-Sep-2024,08:00 AM,06:00 PM,Event,WBO ~ Web3.0 as a Business Opportunity,ZAC1D6327,WBO,SCH24-Not Specified-WBO-00001
1119,NaN,NaN,NaN,14-Nov-2024,08:00 AM,06:00 PM,AdHoc,XHS ~ Marketing with Xiao Hong Shu (XHS): Stra...,NaN,XHS,SCH24-Not Specified-XHS-00003


In [87]:
catering_df = clash_resolved_df.copy()
catering_df['Time of Booking From'].value_counts()

Time of Booking From
08:00 AM    974
06:00 PM    135
05:00 PM      5
12:30 PM      4
04:00 PM      1
01:00 PM      1
02:00 PM      1
Name: count, dtype: int64

In [88]:
# Lists of venues that require catering and catering areas
requires_catering = ['SOE/SCIS2 CTE Scape', 'SOE/SCIS2 Seminar Room 5-1', 'SOE/SCIS2 Seminar Room 5-2']
catering_venue = ['SOE/SCIS2 Catering Area 4A', 'SOE/SCIS2 Catering Area 4B']

# Step 1: Create a dictionary to keep track of booked catering areas for each day
booked_catering = {}

# Step 2: Define the function to populate catering area and avoid clashes
add_catering_rows = []

# Step 3: Define a function to populate the catering area, making sure to not assign catering venues for night sessions
def populate_catering_area(row):
    if row['Facility Name'] in requires_catering:
        # Check if the booking starts at 6 PM or later, and exclude that row
        time_of_booking_from = pd.to_datetime(row['Time of Booking From'], format='%I:%M %p').time()
        if time_of_booking_from >= pd.to_datetime('6:00 PM', format='%I:%M %p').time():
            return row

        date_of_booking = row['Date of Booking']
        
        # Find an available catering venue for the given date
        available_venue = None
        #If a catering area has not been booked for the date, assign one
        for venue in catering_venue:
            if date_of_booking not in booked_catering:
                booked_catering[date_of_booking] = set()
                
            #If a particular catering has already been booked for the date, assign the other venue   
            if venue not in booked_catering[date_of_booking]:
                available_venue = venue
                booked_catering[date_of_booking].add(venue)
                break
        
        if available_venue:
            building = ''  # Assuming building and floor details are not provided in the lists
            floor = ''
            time_of_booking_to = row['Time of Booking To']
            use_type = row['Use Type']
            purpose = row['Purpose']
            event_code = row['Event Code']
            course_code = row['Course Code']
            sch = row['Sch #']
            
            # Create a new row as a list
            catering_row = [available_venue, building, floor, date_of_booking, row['Time of Booking From'],
                            time_of_booking_to, use_type, purpose, event_code, course_code, sch]
            
            # Append the new row to the list of rows to add
            add_catering_rows.append(catering_row)
    
    return row

# Step 4: Apply the function to the DataFrame
catering_df = catering_df.apply(populate_catering_area, axis=1)

# Step 5: Append the new rows to the DataFrame using pd.concat
if add_catering_rows:
    new_rows_df = pd.DataFrame(add_catering_rows, columns=cleaned_df.columns)
    final_catering_df = pd.concat([cleaned_df, new_rows_df], ignore_index=True)
else:
    final_catering_df = cleaned_df

# Check the number of rows that have been added
print(f"Number of current rows: {final_catering_df.shape[0]}")

Number of current rows: 1129


### 9.4 Populate "Venue Preference" Column

1. First, we read all the sheets of the different pillars into separate dataframes, and then we combine into a singular dataframe
2. Then, generate the "Preference Type" column, which comes from the "Room Type" column in the excel sheet by defining a secondary function
3. Next, using this function, we define another function to check if the string "Seminar Room" or "Classroom" exists both in the Preference Type column and the Facility Name column
4. Whether there is a match or not will be indicated in a newly-generated "Venue Preference" column
5. If courses don't have any preference indicated, then it will be indicated as "Preference not indicated" 

In [89]:
preference_check_df = final_catering_df

#Read all sheets in SMUA Venue Matters into separate dataframes
bm_pref = pillar_room_pref_df['Rm Pref - BM']
fit_pref = pillar_room_pref_df['Rm Pref - FIT']
hcml_pref = pillar_room_pref_df['Rm Pref - HCML']
sobi_pref = pillar_room_pref_df['Rm Pref - SOBI']

fit_pref.rename(columns = {'Room Type\n(Seminar Room or Class Room)':'Room Type'}, inplace = True)




In [90]:
#Get the number of rows in each sheet
bm_count = bm_pref.shape[0]
fit_count = fit_pref.shape[0]
hcml_count = hcml_pref.shape[0]
sobi_count = sobi_pref.shape[0]

print("Number of rows in BM Pref: ", bm_pref.shape[0])
print("Number of rows in FIT Pref: ", fit_pref.shape[0])
print("Number of rows in HCML Pref: ", hcml_pref.shape[0])
print("Number of rows in SOBI Pref: ", sobi_pref.shape[0])



Number of rows in BM Pref:  97
Number of rows in FIT Pref:  220
Number of rows in HCML Pref:  88
Number of rows in SOBI Pref:  116


In [91]:
#concatenate all the sheets together
merged_pref = pd.concat([bm_pref, fit_pref, hcml_pref, sobi_pref], ignore_index=True)
print("The total number of rows: ", (bm_count + fit_count + hcml_count + sobi_count))

merged_pref

The total number of rows:  521


,Course Title,Course Code,Room Type\n(Seminar Room or Class Room),Any Comments,Unnamed: 4,Room Type,Any Comments\n
0,Transforming Enterprises Module 1: Experience ...,TEM1-2024,Seminar Room/Classroom,Preference: YPHSL\nRoom Booking until 6pm for ...,NaN,NaN,NaN
1,Transforming Enterprises Module 3: Manage (Eng...,TEM3-2024,Seminar Room/Classroom,Preference: YPHSL\nRoom Booking until 6pm for ...,NaN,NaN,NaN
2,Advanced Data Analytics: Making Better Custome...,ADAMBCDUA,Seminar Room,Seminar Room is a must as the course require p...,NaN,NaN,NaN
3,Applying Smart Tech in a Smart Way: The Right ...,ASTSWTRADT,Classroom,Cluster set up,NaN,NaN,NaN
4,Art & Science of Sales Management,ASSM,Seminar Room/Classroom,Room size for at least 30 pax and to be in clu...,NaN,NaN,NaN
...,...,...,...,...,...,...,...
516,Artificial Intelligence (AI) in Marketing: The...,AIIM,Seminar Room,NaN,NaN,NaN,"Prefer SOE SR5-1, SR4-1"
517,(UOB) Communicating Data with Impact: Data Sto...,CDIDSV,Classroom,NaN,NaN,NaN,"Require classroom at Business school, furnitur..."
518,Community Learning for Personal Development - ...,CLPDMALSETC,Classroom,NaN,NaN,NaN,Preference: SOA Classrooms. Classroom to be se...
519,Community Building and Leadership,CBAL,Classroom,NaN,NaN,NaN,Preference: SOA Classrooms. Classroom to be se...


1. Define a function to extract the course code in merged_pref, then match it with the course code in preference_check_df.
2. Apply the function to the Purpose column to generate the Preference Type column


In [92]:
def populate_venue_match(course_title):
    # Extract the course code by removing everything after "~"
    course_code = course_title.split('~')[0]
    course_code = course_code.strip()
    
    # Get rows where Course Code matches
    matching_rows = merged_pref[merged_pref['Course Code'] == course_code]
    
    # Check if the DataFrame is not empty
    if not matching_rows.empty:
        first_row = matching_rows.iloc[0]
        pref_type = first_row['Room Type']
        return pref_type
    return ""

preference_check_df['Preference Type'] =  preference_check_df['Purpose'].apply(populate_venue_match)

In [93]:
preference_check_df['Preference Type'].unique()

array([nan, '', 'Seminar Room/Classroom', 'Classroom', 'Seminar Room '],
      dtype=object)

In [94]:
#Define a function to check if the venue matches the preference
def check_venue_match(row):
    # Extract the course code from the 'Purpose' column
    course_code = row['Purpose'].split('~')[0].strip()

    # Check if the course code exists in merged_pref
    if course_code not in merged_pref['Course Code'].values:
        return "Preference not indicated"

    preference = str(row['Preference Type']).strip()
    current_venue = str(row['Facility Name']).strip()
    
    sr_condition = ('Seminar Room' in preference) and ('Seminar Room' in current_venue)
    cr_condition = ('Classroom' in preference) and ('Classroom' in current_venue)
    
    if sr_condition or cr_condition:
        return "Match"
    else:
        return "Not Match"
    
preference_check_df['Venue Preference'] = preference_check_df.apply(check_venue_match, axis=1)

In [95]:
#Check the results
preference_check_df.sample(20)

,Facility Name,Building,Floor,Date of Booking,Time of Booking From,Time of Booking To,Use Type,Purpose,Event Code,Course Code,Sch #,Preference Type,Venue Preference
437,NaN,NaN,NaN,27-Dec-2024,12:30 PM,05:00 PM,Event,BIBRBRUT ~ Business Innovation: Building Renew...,ZAC1D2235,BIBRBRUT,SCH24-Not Specified-BIBRBRUT-00002,Seminar Room/Classroom,Not Match
68,NaN,NaN,NaN,19-Oct-2024,08:00 AM,06:00 PM,Event,ACCSMM1 ~ Advanced Certificate in Competitive ...,ZAC1A1134,ACCSMM1,SCH23-Not Specified-ACCSMM1-00002,NaN,Not Match
244,NaN,NaN,NaN,30-Dec-2024,08:00 AM,05:00 PM,Event,ACSCIM2 ~ Advanced Certificate in Supply Chain...,ZAC1D6337,ACSCIM2,SCH24-Not Specified-ACSCIM2-00001,,Preference not indicated
785,NaN,NaN,NaN,29-Oct-2024,08:00 AM,06:00 PM,Event,GCSDTFM2 ~ Graduate Certificate in Strategic D...,ZAC1D2106,GCSDTFM2,SCH24-Not Specified-GCSDTFM2-00002,Seminar Room/Classroom,Not Match
423,NaN,NaN,NaN,14-Nov-2024,08:00 AM,05:00 PM,AdHoc,BBEDEIB ~ Beyond Barriers: Embracing the Power...,NaN,BBEDEIB,SCH24-Not Specified-BBEDEIB-00004,,Preference not indicated
575,NaN,NaN,NaN,28-Oct-2024,08:00 AM,05:00 PM,Event,CSRMFP ~ Cyber Security Risk Management for Fi...,ZAC1D2037,CSRMFP,SCH24-Not Specified-CSRMFP-00002,NaN,Not Match
988,SOA Seminar Room 2-5,School of Accountancy,Level 2,30-Aug-2024,06:00 PM,10:30 PM,Event,PCIMERM4 ~ Professional Certificate in Immersi...,ZAC1A1172,PCIMERM4,SCH24-Not Specified-PCIMERM4-00002,NaN,Not Match
319,NaN,NaN,NaN,27-Sep-2024,08:00 AM,06:30 PM,Event,ACSSBM5 ~ Module 5: Sustainable Finance And Im...,ZAC1E0305,ACSSBM5,SCH23-Not Specified-ACSSBM5-00005,NaN,Not Match
247,NaN,NaN,NaN,27-Sep-2024,08:00 AM,05:00 PM,AdHoc,ACSCIM2 ~ Advanced Certificate in Supply Chain...,NaN,ACSCIM2,SCH24-Not Specified-ACSCIM2-00002,,Preference not indicated
1052,NaN,NaN,NaN,23-Sep-2024,08:00 AM,06:00 PM,AdHoc,SLTIGCALPSM6 ~ Graduate Certificate in Allied ...,NaN,SLTIGCALPSM6,SCH24-Not Specified-SLTIGCALPSM6-00001,,Preference not indicated


### 9.5 Populate "No.of Course Days" Column 

1. Courses that run for 1, 2 or 3 days will be indicated as such in the No. of Course Days column
2. If they run for more than 3 days, it will be indicated as "Above 3 Days"
3. The dataframe will be sorted in Microsoft Excel once uploaded, to prioritise venue bookings for longer running-courses

In [96]:
# Copy the DataFrame
no_course_days_df = preference_check_df

# Initialize an empty dictionary
course_days_dict = {}

# Define a function to build the dictionary
def makedict_course_days(row):
    # Get the schedule identifier
    schedule = row['Sch #']
    
    # If the schedule is not already in the dictionary, add it with the date of booking
    if row['Sch #'] not in course_days_dict:
        course_days_dict[schedule] = row["Date of Booking"] + '@'
    # If the schedule is already in the dictionary, append the date of booking to the existing value
    else:
        course_days_dict[schedule] += row["Date of Booking"] + '@'

# Apply the function to each row of the DataFrame
preference_check_df.apply(makedict_course_days, axis=1)

#Define a function populate the No.of Course Days column based on the length of the start and end date of courses
def populate_days_column(row):
    
    schedule = row['Sch #']
    booking_dates_str = course_days_dict[schedule]
    
    list_booking_dates = booking_dates_str.split('@')
    filtered_booking_dates = [date.strip() for date in list_booking_dates if date.strip()]
    
    # Convert to a set to get unique dates
    unique_dates = list(set(filtered_booking_dates))

    # Count the number of unique dates
    number_of_unique_days = len(unique_dates)
    
    if number_of_unique_days == 1:
        return '1 day'
    elif number_of_unique_days == 2:
        return '2 days'
    elif number_of_unique_days == 3:
        return '3 days'
    elif number_of_unique_days > 3:
        return 'Above 3 days'
    
    
preference_check_df['No. Of Course Days'] = preference_check_df.apply(populate_days_column, axis=1)


In [97]:
preference_check_df.reset_index(drop=True)


,Facility Name,Building,Floor,Date of Booking,Time of Booking From,Time of Booking To,Use Type,Purpose,Event Code,Course Code,Sch #,Preference Type,Venue Preference,No. Of Course Days
0,NaN,NaN,NaN,11-Sep-2024,08:00 AM,06:00 PM,Event,ACABPMM2 ~ Advanced Certificate in Agile Busin...,ZAC1C50073,ACABPMM2,SCH24-Not Specified-ACABPMM2-00001,NaN,Not Match,2 days
1,NaN,NaN,NaN,12-Sep-2024,08:00 AM,06:00 PM,Event,ACABPMM2 ~ Advanced Certificate in Agile Busin...,ZAC1C50073,ACABPMM2,SCH24-Not Specified-ACABPMM2-00001,NaN,Not Match,2 days
2,NaN,NaN,NaN,19-Aug-2024,08:00 AM,06:00 PM,Event,ACABPMM3 ~ Advanced Certificate in Agile Busin...,ZAC1C50074,ACABPMM3,SCH24-Not Specified-ACABPMM3-00001,NaN,Not Match,3 days
3,NaN,NaN,NaN,20-Aug-2024,08:00 AM,06:00 PM,Event,ACABPMM3 ~ Advanced Certificate in Agile Busin...,ZAC1C50074,ACABPMM3,SCH24-Not Specified-ACABPMM3-00001,NaN,Not Match,3 days
4,NaN,NaN,NaN,21-Aug-2024,08:00 AM,06:00 PM,Event,ACABPMM3 ~ Advanced Certificate in Agile Busin...,ZAC1C50074,ACABPMM3,SCH24-Not Specified-ACABPMM3-00001,NaN,Not Match,3 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,SOE/SCIS2 Catering Area 4A,,,27-Sep-2024,08:00 AM,06:00 PM,Event,DAW1 ~ Data Analytics @ Work 101,ZAC1D6189,DAW1,SCH24-Not Specified-DAW1-00004,Seminar Room/Classroom,Not Match,2 days
1125,SOE/SCIS2 Catering Area 4A,,,04-Nov-2024,08:00 AM,06:00 PM,Event,DAW1 ~ Data Analytics @ Work 101,ZAC1D6189,DAW1,SCH24-Not Specified-DAW1-00005,Seminar Room/Classroom,Not Match,2 days
1126,SOE/SCIS2 Catering Area 4A,,,05-Nov-2024,08:00 AM,06:00 PM,Event,DAW1 ~ Data Analytics @ Work 101,ZAC1D6189,DAW1,SCH24-Not Specified-DAW1-00005,Seminar Room/Classroom,Not Match,2 days
1127,SOE/SCIS2 Catering Area 4A,,,21-Aug-2024,08:00 AM,05:00 PM,Event,MW3 ~ The Metaverse and Web 3.0,ZAC1D6318,MW3,SCH24-Not Specified-MW3-00002,Seminar Room,Not Match,2 days


### 9.6 Course Code cannot be found in FBS Report

The course in gvSession may not be found in the FBS Report for the following reasons:

1. It is a new course, hence no previous bookings were made
2. The course did not run in the previous period.

Hence, the "Facility Name" Column values for these courses will be indicated as "Assignment of Venues Required"

In [98]:
preference_check_df['Facility Name'].fillna("Assignment of venue required", inplace=True)

preference_check_df.sample(10)

,Facility Name,Building,Floor,Date of Booking,Time of Booking From,Time of Booking To,Use Type,Purpose,Event Code,Course Code,Sch #,Preference Type,Venue Preference,No. Of Course Days
833,SCIS1 Classroom 3-1,School of Computing & Information Systems 1,Level 3,29-Aug-2024,08:00 AM,05:00 PM,Event,HMEPR ~ Hedging & Management of Energy Price Risk,ZAC1D6020,HMEPR,SCH24-Not Specified-HMEPR-00007,Seminar Room/Classroom,Match,3 days
849,Assignment of venue required,NaN,NaN,23-Sep-2024,08:00 AM,06:00 PM,Event,ICCTGC2 ~ Innovation Culture Catalyst (ICC): T...,ZAC1D2100,ICCTGC2,SCH24-Not Specified-ICCTGC2-00002,Seminar Room,Not Match,2 days
749,Assignment of venue required,NaN,NaN,24-Oct-2024,08:00 AM,06:00 PM,Event,GCDHM3 ~ Graduate Certificate in Digital Healt...,ZAC1C50018,GCDHM3,SCH24-Not Specified-GCDHM3-00001,NaN,Not Match,3 days
1091,Assignment of venue required,NaN,NaN,18-Sep-2024,08:00 AM,06:00 PM,Event,TSMESSD ~ Transforming Small and Medium Enterp...,ZAC1E2702,TSMESSD,SCH24-Not Specified-TSMESSD-00002,NaN,Not Match,Above 3 days
281,Assignment of venue required,NaN,NaN,20-Sep-2024,08:00 AM,06:00 PM,Event,ACSCITM3 ~ Advanced Certificate in Supply Chai...,ZAC1D2202,ACSCITM3,SCH24-Advanced Diploma-ACSCITM3-00003,,Preference not indicated,3 days
952,Assignment of venue required,NaN,NaN,23-Oct-2024,08:00 AM,06:00 PM,Event,MMFA ~ Macro Market Fundamental Analysis,ZAC1D6069,MMFA,SCH24-Not Specified-MMFA-00002,Seminar Room/Classroom,Not Match,2 days
391,Assignment of venue required,NaN,NaN,13-Nov-2024,08:00 AM,06:00 PM,Event,ADAMBCDUA ~ Advanced Data Analytics: Making Be...,ZAC1E1012,ADAMBCDUA,SCH24-Advanced Diploma-ADAMBCDUA-00001,NaN,Not Match,3 days
435,Assignment of venue required,NaN,NaN,05-Oct-2024,08:00 AM,06:00 PM,AdHoc,BDSJCBE ~ Behavioural Decision Science: Judgme...,NaN,BDSJCBE,SCH24-Not Specified-BDSJCBE-00002,,Preference not indicated,3 days
784,Assignment of venue required,NaN,NaN,28-Oct-2024,08:00 AM,06:00 PM,Event,GCSDTFM2 ~ Graduate Certificate in Strategic D...,ZAC1D2106,GCSDTFM2,SCH24-Not Specified-GCSDTFM2-00002,Seminar Room/Classroom,Not Match,2 days
526,Assignment of venue required,NaN,NaN,15-Nov-2024,06:00 PM,10:30 PM,Event,CFTRMVO(CR) ~ Certificate in Financial Trading...,ZAC1A1048,CFTRMVO(CR),SCH23-Advanced Diploma-CFTRMVO(CR)-00006,NaN,Not Match,3 days


### 9.7 Final Check on Facility Names

1. Check if there are any facility names that are not in suitable FBS format, and use Facility Names to change those names accordingly

In [99]:
def update_facility_name(row, mapping):
  """
  This function updates the facility name based on the TMS name mapping.

  Args:
      row (pd.Series): A row from the dataframe.
      mapping (pd.DataFrame): Dataframe containing TMS name to FBS name mapping.

  Returns:
      str: The updated facility name.
  """
  tms_name = row['Facility Name']
  if tms_name in mapping['TMS_Facility Name'].tolist():
    return mapping.loc[mapping['TMS_Facility Name'] == tms_name, 'Converted FBS naming'].tolist()[0]
  else:
    return tms_name  # Keep the original facility name if not found in mapping

# Update facility names based on TMS name mapping
preference_check_df['Facility Name'] = preference_check_df.apply(update_facility_name, axis=1, args=(facility_names_df,))

## 10.0 Final Touches

Upon having the dataframe version of the populated Facility Booking List. We will be using it to populate the values in the actual .xlsx file. 

In this section, we will be, 

1. Dropping any unnecessary columns and retaining only those columns required in the FBS Template, plus the additional columns
2. Change the "Time of Booking From" and "Time of Booking To" columns to 24HR format, and rename to required format by FBS
3. Generate the "Any Comments" column
4. Revert back the "Course Code" column to be empty, as FBS does not recognise it 
5. Renaming of some column headers to be equivalent to what is inside the template
6. Sorting the dataframe to group courses together by the Purpose and Date of Booking
7. Changing all column values to text

### 10.1 Dropping Unnecessary Columns and Retaining Only Those Necessary in FBS

In [100]:
#Checking if there are any missing values in the Facility Name column
print(f"Number of missing values in the Facility Name Column: {preference_check_df['Facility Name'].isnull().sum()}")

#Drop any unnecessary columns
final_df = preference_check_df[['Facility Name', 'Building', 'Floor', 'Date of Booking', 'Time of Booking From', 'Time of Booking To', 'Use Type', 'Purpose', 'Event Code', 'Course Code', 'Sch #', 'No. Of Course Days', 'Venue Preference']]

#Generating the "Any Comments"
final_df['Any Comments'] = np.nan

#Revert back the "Course Code" column to be empty, as it is not required for FBS 
final_df['Course Code'] = np.nan


Number of missing values in the Facility Name Column: 0


### 10.2 **(OPTIONAL)** Populating the "Building" and "Floor" columns

Comment out this code if required to populate "Building" and "Floor" columns with values, if not, by default, these columns will be empty

In [101]:
final_df['Building'] = np.nan
final_df['Floor'] = np.nan

### 10.3 Renaming of Date and Time Columns to Fit Naming Convention of FBS Booking Template 

In [102]:
# Convert 'Time of Booking From' and 'Time of Booking To' to 24-hour format
final_df['Time of Booking From'] = pd.to_datetime(final_df['Time of Booking From'], format='%I:%M %p').dt.strftime('%H:%M')
final_df['Time of Booking To'] = pd.to_datetime(final_df['Time of Booking To'], format='%I:%M %p').dt.strftime('%H:%M')

# Standardize Date and Time Column names
final_df.rename(columns = {'Date of Booking':'Date of Booking (dd-MMM-yyyy)',
                           'Time of Booking From':'Time of Booking From (HH:mm)',
                           'Time of Booking To':'Time of Booking To (HH:mm)'}, inplace = True)

In [103]:
#Replace "" values with null for easier accounting purposes
final_df.replace("", np.nan, inplace=True)
final_df.replace("nan", np.nan, inplace=True)

In [104]:
final_df.isna().sum()

Facility Name                       0
Building                         1129
Floor                            1129
Date of Booking (dd-MMM-yyyy)       0
Time of Booking From (HH:mm)        0
Time of Booking To (HH:mm)          0
Use Type                            0
Purpose                             0
Event Code                        270
Course Code                      1129
Sch #                               0
No. Of Course Days                  0
Venue Preference                    0
Any Comments                     1129
dtype: int64

### 10.4 Sort the Dataframe by "Purpose" and "Date of Booking" to group courses together

In [105]:
final_df = final_df.sort_values(by=['Purpose', 'Date of Booking (dd-MMM-yyyy)']).reset_index(drop=True)

### 10.5 Standardize all the column values to text format

In [106]:
# Convert all columns to string format
final_df = final_df.astype(str)

# Replace 'nan' with empty string ('') for all columns with empty values
final_df = final_df.replace('nan', '')

# Function to check if every column in the DataFrame is in text format
def check_text_columns(df):
    non_text_columns = []
    for column in df.columns:
        if not df[column].apply(lambda x: isinstance(x, str)).all():
            non_text_columns.append(column)
    
    if non_text_columns:
        print("The following columns are not entirely in text format:")
        for col in non_text_columns:
            print(col)
    else:
        print("All columns are in text format.")

# Call the function
check_text_columns(final_df)

All columns are in text format.


In [107]:
#Check the final dataframe
final_df

,Facility Name,Building,Floor,Date of Booking (dd-MMM-yyyy),Time of Booking From (HH:mm),Time of Booking To (HH:mm),Use Type,Purpose,Event Code,Course Code,Sch #,No. Of Course Days,Venue Preference,Any Comments
0,Assignment of venue required,,,11-Sep-2024,08:00,18:00,Event,ACABPMM2 ~ Advanced Certificate in Agile Busin...,ZAC1C50073,,SCH24-Not Specified-ACABPMM2-00001,2 days,Not Match,
1,Assignment of venue required,,,12-Sep-2024,08:00,18:00,Event,ACABPMM2 ~ Advanced Certificate in Agile Busin...,ZAC1C50073,,SCH24-Not Specified-ACABPMM2-00001,2 days,Not Match,
2,Assignment of venue required,,,19-Aug-2024,08:00,18:00,Event,ACABPMM3 ~ Advanced Certificate in Agile Busin...,ZAC1C50074,,SCH24-Not Specified-ACABPMM3-00001,3 days,Not Match,
3,Assignment of venue required,,,20-Aug-2024,08:00,18:00,Event,ACABPMM3 ~ Advanced Certificate in Agile Busin...,ZAC1C50074,,SCH24-Not Specified-ACABPMM3-00001,3 days,Not Match,
4,Assignment of venue required,,,21-Aug-2024,08:00,18:00,Event,ACABPMM3 ~ Advanced Certificate in Agile Busin...,ZAC1C50074,,SCH24-Not Specified-ACABPMM3-00001,3 days,Not Match,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,Assignment of venue required,,,14-Oct-2024,08:00,18:00,Event,WA1AWMPP ~ Workplace Automation 101: Automatio...,ZAC1D6187,,SCH24-Not Specified-WA1AWMPP-00002,2 days,Not Match,
1125,Assignment of venue required,,,15-Oct-2024,08:00,18:00,Event,WA1AWMPP ~ Workplace Automation 101: Automatio...,ZAC1D6187,,SCH24-Not Specified-WA1AWMPP-00002,2 days,Not Match,
1126,Assignment of venue required,,,30-Sep-2024,08:00,18:00,Event,WBO ~ Web3.0 as a Business Opportunity,ZAC1D6327,,SCH24-Not Specified-WBO-00001,1 day,Not Match,
1127,Assignment of venue required,,,14-Nov-2024,08:00,18:00,AdHoc,XHS ~ Marketing with Xiao Hong Shu (XHS): Stra...,,,SCH24-Not Specified-XHS-00003,1 day,Preference not indicated,


## 11.0 Uploading the Dataframe into "Facility Booking List" Excel Workbook

1. Specify the output file path, the name of the sheet you want to upload to, and the sheet name which contains the template for the headers
2. This code will copy the template sheet's headers into the new sheet, such that only the values from the final dataframe will be appended to their respective columns. 
3. Additional columns will have to formatted within the Excel File itself

In [108]:
# Specify the file paths
output_file_path = 'Facility+Booking+List.xlsx'
sheet_name = '01. Facility Booking List'
new_sheet_name = 'Final Facility Booking List'

# Load the template workbook and select the template sheet
wb = load_workbook(output_file_path)

# Check if the new sheet already exists and delete it if it does
if new_sheet_name in wb.sheetnames:
    del wb[new_sheet_name]

template_sheet = wb[sheet_name]

# Copy the template sheet to retain the formatting
wb.copy_worksheet(template_sheet)
copied_sheet = wb[sheet_name + ' Copy']
copied_sheet.title = new_sheet_name

# Append DataFrame values to the new sheet
start_row = 2  # Assuming the first row is the header row

# Write headers for the additional columns
for col_idx, col_name in enumerate(final_df.columns, start=1):
    copied_sheet.cell(row=1, column=col_idx, value=col_name)

# Write data
for r_idx, row in enumerate(dataframe_to_rows(final_df, index=False, header=False), start=start_row):
    for c_idx, value in enumerate(row, start=1):
        copied_sheet.cell(row=r_idx, column=c_idx, value=value)

# Auto-fit column width
for col in copied_sheet.columns:
    max_length = 0
    column = get_column_letter(col[0].column)
    for cell in col:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2)
    copied_sheet.column_dimensions[column].width = adjusted_width

# Save the modified workbook
wb.save(output_file_path)
print("Facility Booking List has been populated with the necessary values and uploaded into Excel File")

Facility Booking List has been populated with the necessary values and uploaded into Excel File
